# Clean and reformat CDR data from its multi-tab excel file into a single csv

### About the data

CDR data is tricky -- the form used by law enforcement has changed over time, first in 2005, then again in 2016. The data before 2005 is known to be be sparse and poorly enforced, so we ignore those entries. The 2005 and 2016 versions of the form have some overlap and some differences, so we must be careful in how we merge them.

Further, some older deaths that happened in an inpatient setting are not reported in these dataset. In the past, for the first version of the TJI website, these records were requested. We have an "old master file" that was used in the first TJI website that includes these deaths, but also many others. We need to load this file and include any records that we don't yet have.

**In this repo you can find blank versions of the [2005](https://github.com/texas-justice-initiative/data-processing/blob/master/forms/CDR%20Form%20Version%202005.pdf) and [2016](https://github.com/texas-justice-initiative/data-processing/blob/master/forms/CDR%20Form%20Version%202016.pdf) forms, to see for yourself exactly what fields are collected and how.**

### Datasets used


* Input:
  * `tji/raw-and-processing/CDR - All Reports.xlsx`
  * `tji/raw-and-processing/reformatted_cdr_2017_master_file`
  * `tji/auxiliary-datasets/agencies_and_counties`
* Output:
  * `tji/deaths-in-custody/cleaned_custodial_death_reports.csv`
  
##### Author: Everett Wetchler (everett.wetchler@gmail.com)

## STEPS

**1. Setup and read data**
- 1a. Configuration and imports
    - Constants
    - Libraries
- 1.b Read data
    - Drop records with death date pre-2005
- 1c. Helper methods/code
    

**2. Join disparate datasets, rename and reformat columns**
- 2a. Reduce columns used and rename
    - Drop columns that we either don't need or haven't written code to clean yet
    - Rename columns to be more machine friendly
- 2b. Add rows from the old master file to get BJS inpatient deaths 
    - Merge the "old master" file with the other CDR records
- 2c. Column type conversions
    - Convert date columns
    - Float columns covert from string
    - Upcase string columns
    - Add `death_date` column
    
**3. Begin Deep Cleaning**
- 3a. Standardize `race` values
- 3b. Standardize agency information
- 3c. Fix death-information related columns
    - Fix `Death Location`
    - fix `Means of Death`
    - Fix `manner_of_death` and `manner of death`
    - Fix `pre_existing_medical_condition`
    - Fix `who_caused_the_death`
    - Fix `medical_examinor_coroner_evalution`
- 3d. Fix other columns one-by-one
    - Standardize `gender`
    - Fix `were_there_charges`
    - Fix `type_of_custody`
    - Fix `specific_type_of_custody_facility`
- 3e. Identify and drop a range of unnecessary columns
    - `entry_date_time_n_a` and `custody_date_na`
    - `department_type`

**4. Deduplication**
- 4a. Whole row duplicates
- 4b. Define some dedup functions
- 4c. Dedup
- 4d. Inspect results

**5. Add new columns, rename/reorder, otherwise finalize dataframe**
- 5a. New columns and bookkeeping columns
    - Column for the time (in days) between incarceration (or incident) and death
- 5b. Rename and reorder columns sensibly
- 5c. Final look before writing


**6. Write**


**7. TODOs -- problems that still exist in the data and need work**

---
Let's begin...

<div style="background-color: #080;height: 20px"></div>

# 1. Setup and read data

<div style="background-color: #008;height: 20px"></div>

## 1a. Configuration and imports

In [116]:
# Constants so we don't have to hunt through the code to tweak these in many places

DW_PROJECT_CDR = 'tji/deaths-in-custody'
DW_PROJECT_OIS = 'tji/officer-involved-shootings'
DW_PROJECT_AUXILIARY_DATASETS = 'tji/auxiliary-datasets'
DW_PROJECT_RAW_AND_PROCESSING = 'tji/raw-and-processing'

OUTPUT_DW_PROJECT = DW_PROJECT_CDR
OUTPUT_FILENAME = 'cleaned_custodial_death_reports.csv'

In [117]:
# Import ALL the things

import os
import sys
import json
import boto3
import datetime
import numpy as np
import pandas as pd
import datadotworld as dw
import pygsheets

from io import StringIO
from lib.cleaning_tools import *

sys.path.append(os.getcwd() + '/../data_cleaning')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%load_ext watermark
%watermark -a "Everett Wetchler, Aiden Yang" -d -t -z -w -p numpy,pandas,datadotworld

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Everett Wetchler, Aiden Yang 2020-10-25 10:26:45 MDT 

numpy 1.17.2
pandas 1.1.2
datadotworld 1.7.0
watermark 2.0.2


In [118]:
# Custom libraries specific to this project
from lib.standardize_police_agency_names import standardize_agency_name

<div style="background-color: #008;height: 20px"></div>

## 1.b Read data and take a glance

In [119]:
datasets = dw.load_dataset(DW_PROJECT_AUXILIARY_DATASETS, force_update=True)
agency_county = datasets.dataframes['agencies_and_counties']
agency_county = agency_county.set_index('agency')['county'].to_dict()

In [120]:
gc = pygsheets.authorize(service_file='/Users/hlukas/Downloads/client_secret_service.json')

In [121]:
gc.drive.enable_team_drive('0ACeQWapAwOLqUk9PVA')
titles = gc.spreadsheet_titles()
ids = gc.spreadsheet_ids()

cdr_id = [ids[index] for index in range(len(titles)) if titles[index] == 'CDR Reports All'][0]
output = gc.open_by_key(cdr_id)

dfs = {}

# drop empty columns or the concatenation does not work
for sheet_name in ['Form Version 2005', 'Form Version 2016', 'Older Forms']:
    sheet = [i for i in range(4) if output.worksheets()[i].title == sheet_name][0]
    wk = output[sheet]
    try:
        dfs[sheet_name] = wk.get_as_df(include_tailing_empty = True).drop(columns = [''], axis= 1)
    except KeyError:
        dfs[sheet_name] = wk.get_as_df(include_tailing_empty = True)
    
dfs['Form Version 2005']['form_version'] = 'V_2005'
dfs['Form Version 2016']['form_version'] = 'V_2016'
dfs['Older Forms']['form_version'] = 'V_OLDER'

cdr = pd.concat([dfs['Form Version 2005'], dfs['Form Version 2016']])
print('Read %d raw CDR records. Ignoring %d from older form versions (pre-2005).' % (
    sum(len(x) for x in dfs), len(dfs['Older Forms'])))
print('Keeping %d records using form version 2005, and %d using version 2016 (keeping %d in total)' % (
    len(dfs['Form Version 2005']), len(dfs['Form Version 2016']), len(cdr)))

Read 45 raw CDR records. Ignoring 5468 from older form versions (pre-2005).
Keeping 5156 records using form version 2005, and 4231 using version 2016 (keeping 9387 in total)


In [122]:
# datasets = dw.load_dataset(DW_PROJECT_AUXILIARY_DATASETS, force_update=True)
# agency_county = datasets.dataframes['agencies_and_counties']
# agency_county = agency_county.set_index('agency')['county'].to_dict()

In [123]:
# dfs = read_dtw_excel(DW_PROJECT_RAW_AND_PROCESSING, 'original/CDR Reports All.xlsx')
# dfs['Form Version 2005']['form_version'] = 'V_2005'
# dfs['Form Version 2016']['form_version'] = 'V_2016'
# dfs['Older Forms']['form_version'] = 'V_OLDER'
# cdr = pd.concat([dfs['Form Version 2005'], dfs['Form Version 2016']])
# print('Read %d raw CDR records. Ignoring %d from older form versions (pre-2005).' % (
#     sum(len(x) for x in dfs), len(dfs['Older Forms'])))
# print('Keeping %d records using form version 2005, and %d using version 2016 (keeping %d in total)' % (
#     len(dfs['Form Version 2005']), len(dfs['Form Version 2016']), len(cdr)))

### In theory, all these records should be for deaths in 2005 or later. Let's double check and drop any miscreants.

In [124]:
assert cdr['Death Date and Time'].isnull().sum() == 0

In [125]:
cdr['Death Date and Time'] = pd.to_datetime(cdr['Death Date and Time'])


In [126]:
before = len(cdr)
cdr = cdr[cdr['Death Date and Time'].dt.year >= 2005]
after = len(cdr)
print('Dropped %d (of %d) reports for deaths before 2005, leaving %d' % (before - after, before, after))

Dropped 6 (of 9387) reports for deaths before 2005, leaving 9381


In [127]:
print("Data goes from %s to %s" % (cdr['Death Date and Time'].min().strftime("%Y-%m-%d"),
                                   cdr['Death Date and Time'].max().strftime("%Y-%m-%d")))

Data goes from 2005-01-01 to 2020-10-06


### A quick look at the raw data

In [128]:
dfs['Form Version 2005'].head()

,CDR: CDR Name,Report Date,Status,Version Type,Version Number,Department Type,Agency Name,Agency Phone,Agency Address,Agency City,Agency County,Agency State,Agency Zip,Director Salutation,Director First Name,Director Middle Name,Director Last Name,Reporter Name Original CDR,Reporter Email,Street Address,City,County,Type of Custody,Specific Type of Custody/Facility,Custody Type Facility,Entry Date Time,Entry Date Time N/A,Death Location,Death Location Elsewhere,First Name,Middle Name,Last Name,Suffix,Date of Birth,Sex,Ethnicity,Ethnicity Other,Death Date and Time,Age At Time Of Death,Date/Time of Custody or Incident,Custody Date NA,Medical Examinor/Coroner Evalution?,Manner of Death,Manner of Death Description,Death Reason,Medical Cause of Death,Medical Treatment,Medical Treatment Description,Who caused the death?,Death Causer Other,Type of Death Weapon,Death Weapon Other Description,Pre existing medical condition?,Means of Death,Means of Death Other,Offense 1,Offense 2,Offense 3,Were the Charges:,Type of Offense,"Type of Offense, Other",Injured By,Threaten the officer(s) involved,Resist being handcuffed or arrested?,Try to escape/flee from custody,"Grab, hit or fight with the officer(s)",Other Behavior,Specify Other Behavior,Appear intoxicated (alcohol or drugs),Use weapon threaten/assault officer(s),Entry Behavior,Under Restraint,Type of Restraint,"Other device, specify",form_version
0,PA05001C,3/2/05 14:48,Submitted,ORIGINAL VERSION,1,POLICE,Fort Worth Police Dept.,8178778022,350 W. Belknap,Fort Worth,TARRANT,TX,76102,Chief,Ralph,,Mendoza,Renee Gray,renee.gray@oag.state.tx.us,1509 W. Hammond,Fort Worth,Tarrant,Police Custody (pre-booking),Custody of Peace Officer during/fleeing arrest,,,1,At the crime/arrest scene,,Dino,,Gomez,,4/3/1964,Male,Hispanic,,1/1/05 15:04,40,1/1/05 15:04,0,"Yes, results are available",Justifiable Homicide,,Injuries only,Gunshot Wound to the Chest,Not Applicable,,Law enforcement/correctional staff,,Handgun,,Not Applicable; cause of death was accidental ...,Firearm,,Aggravated Assault,,,Not filed at time of death,,,Injured by Officer,Yes,Yes,Yes,No,0,,No,1,NA,No,,,V_2005
1,PA05002CJ,3/3/05 14:07,Submitted,ORIGINAL VERSION,1,SHERIFF,Travis County Sheriff's Dept.,5128549770,P. O. Box 1748,Austin,TRAVIS,TX,78767,Sheriff,Margo,L.,Frasier,Renee Gray,renee.gray@oag.state.tx.us,3614 Bill Price Road,Del Valle,Travis,County Jail,Jail - multiple occupancy cell,,10/20/04 20:56,0,At medical facility,,Michael,Darnell,Dickson,,3/4/1953,Male,African-American,,1/1/05 15:59,51,10/20/04 20:56,0,"Yes, results are available",Natural Causes/Illness,Cardiac Arrest,Medical condition only (e.g. heart attack),Myocardial infarction,Yes,"Treatment for hypertension, administered Proca...","Not applicable; cause of death was suicide, in...",,Not Applicable,,Pre-existing medical condition,Not applicable; cause of death was intoxicatio...,,Man/Del/Sell/Poss Controlled Substance,,,Filed,,,Injured by NA,No,No,No,No,0,,No,0,Medical,No,,,V_2005
2,PA05003P,3/3/05 14:16,Submitted,ORIGINAL VERSION,1,STAGENCY,Texas Department Of Criminal Justice,9364376716,P.O. Box 4003,Huntsville,TRAVIS,TX,773424003,Mr.,Chris,H.,Stallings,Renee Gray,renee.gray@oag.state.tx.us,21 FM 247,Huntsville,Walker,Penitentiary,TDCJ,Byrd,12/17/04 9:00,0,At medical facility,,Darryl,Glenn,Wallace,,12/3/1960,Male,African-American,,1/2/05 4:35,44,12/17/04 9:00,0,"Yes, results are available",Natural Causes/Illness,Cardiac,Medical condition only (e.g. heart attack),Cardiac death secondary to cardiac ischemia th...,Yes,Nitroglycerin,"Not applicable; cause of death was suicide, in...",,Not Applicable,,Pre-existing medical condition,Not applicable; cause of death was intoxicatio...,,Burglary of Habitation,,,Convicted,,,Injured by NA,No,No,No,No,0,,No,0,NA,No,,,V_2005
3,PA05004C,3/3/05 14:40,Submitted,ORIGINAL VERSION,1,POLICE,San Antonio Police Dept.,2102077449,"214 W. Nueva, Suite 331",San Antonio,BEXAR,TX,78207,Chief,Albert,,Ortiz,Renee Gray,renee.gray@oag.state.tx.us,2102 

In [129]:
dfs['Form Version 2016'].head()

,CDR: CDR Name,Version Type,Version Number,Report Date,Report Time,Status,Agency Name,Agency Address,Agency City,Agency State,Agency Zip,Director Salutation,Director First Name,Director Middle Name,Director Last Name,Reporter Name,Reporter Email,First Name,Middle Name,Last Name,Suffix,Date of Birth,Sex,Race,Age At Time Of Death,Date/Time of Custody or Incident,Death Date and Time,Medical Examinor/Coroner Evalution?,Manner of Death,Manner of Death Description,Medical Cause of Death,Medical Treatment,Who caused the death?,Type of weapon that caused death?,"Other weapon, specify",Pre existing medical condition?,Means of Death,Means of Death Other,Street Address,City,County,Zip,Location Category,Other Location Category,Type of Custody,Specific Type of Custody/Facility,TDCJ - Specify Unit,Entry Date Time,Entry Date Time N/A,Death Location,Death Location Elsewhere,Other Agencies Respond?,Offense 1,Offense 2,Offense 3,Were the Charges:,Type of Offense,"Type of Offense, Other",Decedent display/use of weapons,Decedent Display or Use Weapon Details,Specify Weapon Used,Attempt to Injure Others?,Appear intoxicated (alcohol or drugs),Make suicidal statements?,Exhibit any mental health problems?,Exhibit any medical problems?,Barricade self or initiate standoff?,Resist being handcuffed or arrested?,Physically attempt/assault officer(s),Gain possession of officer's weapon,Verbally threaten other(s) including law,Escape or attempt to escape/flee custody,Attempt gain possession officer's weapon,Under Restraint,Type of Restraint,"Other device, specify",form_version
0,16-3-C,AMENDED,4,12/12/16 13:02,,Submitted,Texas Department Of Public Safety,PO BOX 4087,Austin,TX,78773,Director,Steven,,McCraw,Joanne Scarbrough,joanne.scarbrough@dps.texas.gov,Ivory,Charles,Pantallion,III,9/14/1980,Male,Black or African American,36,11/22/16 9:26,11/22/16 9:26,"Yes, results are available",Homicide (includes Justifiable Homicide),,Multiple Gunshot Wounds,Not Applicable,Law enforcement/correctional personnel,Handgun; Rifle/shotgun,,Not Applicable; cause of death was accidental ...,Firearm,,7300 Interstate 10 W,Baytown,Harris,77521,Roadway/highway/street/sidewalk,,Police Custody (pre-booking),Custody of Law Enforcement Personnel during/fl...,,,1,Scene of incident,,Yes,Aggravated Assault on Peace Officer,Evading Arrest or Detention,,Not filed at time of death,Violent Crime Against Persons,,"Yes, mark all that apply",Displayed firearm without discharge,,Yes (select all that apply),No,No,No,No,Unknown,Yes,Yes,No,Unknown,Yes,No,No,,,V_2016
1,14-1-C,AMENDED,2,12/13/16 16:20,,Submitted,Texas Department Of Public Safety,PO BOX 4087,Austin,TX,78773,Director,Steven,,McCraw,Joanne Scarbrough,joanne.scarbrough@dps.texas.gov,James,Earl,Nicholas,,4/24/1966,Male,Anglo or White,48,9/11/14 17:00,9/11/14 17:00,"Yes, results are available",Could not be determined,,Multiple Gunshot Wounds,Not Applicable,Law enforcement/correctional personnel,"Firearm, unspecified",,Not Applicable; cause of death was accidental ...,Firearm,,2030 Jacintoport Blvd.,Houston,Harris,77015,Roadway/highway/street/sidewalk,,Police Custody (pre-booking),Custody of Law Enforcement Personnel during/fl...,,,1,Scene of incident,,Yes,Capital Murder - Filed,Aggravated Assault on a Peace Officer - Not Filed,Evading Arrest or Detention - Not Filed,Filed,Violent Crime Against Persons,,"Yes, mark all that apply",Discharged firearm,,Yes (select all that apply),No,No,No,No,No,Yes,Yes,No,Unknown,Yes,No,No,,,V_2016
2,16-4-P,ORIGINAL VERSION,1,12/14/16 15:27,,Submitted,TDCJ/Office of the Inspector General,"2503 Lake Road, Suite 5",Huntsville,TX,77340,Other,John,,West,Analou Sievers,analou.sievers@tdcj.texas.gov,Percy,,Froman,,12/19/1969,Male,Anglo or White,46,1/28/00 0:00,11/17/16 8:25,"No, evaluation not planned",Natural,,Colon Cancer,No,Not applicable,Not Applicable,,Pre-existing medical condition,"Not applicable, cause of death was illness/nat...",,8602 Peach Street,Lubbock,Lubbock,79404,Law Enforcement Facility,,Penitentia

<div style="background-color: #008;height: 20px"></div>

## 1c. Helper methods/code

In [130]:
def show_notnull_chart(cdr, vertical=False):
    '''Show which columns have null values, how often, and break down by form version.'''
    versions = sorted(set(cdr.form_version))
    frames = []
    for v in versions:
        fr = cdr[cdr.form_version == v]
        s = fr.notnull().mean()
        frames.append(s)
    frame = pd.concat(frames, axis=1)
    frame.columns = versions
    frame.sort_index(inplace=True)
    if not vertical:
        frame = frame.T
    return frame.style.background_gradient(cmap='RdYlGn', axis=(0 if vertical else 1))

In [131]:
OTHER_SPECIFY = 'OTHER, SPECIFY'

<div style="background-color: #080;height: 20px"></div>

# 2. Join disparate datasets, rename and reformat columns

<div style="background-color: #008;height: 15px"></div>

## 2a. Reduce the number of columns that we keep, and rename them more conveniently

#### Look at how often our columns are populated, broken down by form version (since each ask slightly different questions)

In [132]:
show_notnull_chart(cdr)

,Age At Time Of Death,Agency Address,Agency City,Agency County,Agency Name,Agency Phone,Agency State,Agency Zip,Appear intoxicated (alcohol or drugs),Attempt gain possession officer's weapon,Attempt to Injure Others?,Barricade self or initiate standoff?,CDR: CDR Name,City,County,Custody Date NA,Custody Type Facility,Date of Birth,Date/Time of Custody or Incident,Death Causer Other,Death Date and Time,Death Location,Death Location Elsewhere,Death Reason,Death Weapon Other Description,Decedent Display or Use Weapon Details,Decedent display/use of weapons,Department Type,Director First Name,Director Last Name,Director Middle Name,Director Salutation,Entry Behavior,Entry Date Time,Entry Date Time N/A,Escape or attempt to escape/flee custody,Ethnicity,Ethnicity Other,Exhibit any medical problems?,Exhibit any mental health problems?,First Name,Gain possession of officer's weapon,"Grab, hit or fight with the officer(s)",Injured By,Last Name,Location Category,Make suicidal statements?,Manner of Death,Manner of Death Description,Means of Death,Means of Death Other,Medical Cause of Death,Medical Examinor/Coroner Evalution?,Medical Treatment,Medical Treatment Description,Middle Name,Offense 1,Offense 2,Offense 3,Other Agencies Respond?,Other Behavior,Other Location Category,"Other device, specify","Other weapon, specify",Physically attempt/assault officer(s),Pre existing medical condition?,Race,Report Date,Report Time,Reporter Email,Reporter Name,Reporter Name Original CDR,Resist being handcuffed or arrested?,Sex,Specific Type of Custody/Facility,Specify Other Behavior,Specify Weapon Used,Status,Street Address,Suffix,TDCJ - Specify Unit,Threaten the officer(s) involved,Try to escape/flee from custody,Type of Custody,Type of Death Weapon,Type of Offense,"Type of Offense, Other",Type of Restraint,Type of weapon that caused death?,Under Restraint,Use weapon threaten/assault officer(s),Verbally threaten other(s) including law,Version Number,Version Type,Were the Charges:,Who caused the death?,Zip,form_version
V_2005,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
V_2016,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Drop columns that we either don't need or haven't written code to clean yet.

In [133]:
keep_text = '''Both forms

- Age At Time Of Death
- Agency Address
- Agency City
- Agency Name
- Agency Zip
- CDR: CDR Name
- City
- County
- Date of Birth
- Date/Time of Custody or Incident
- Death Date and Time
- Death Location
- Death Location Elsewhere
- Entry Date Time
- Entry Date Time N/A
- First Name
- Middle Name
- Last Name
- Suffix
- Manner of Death
- Manner of Death Description
- Means of Death
- Means of Death Other
- Medical Cause of Death
- Medical Examinor/Coroner Evalution?
- Medical Treatment
- Offense 1
- Offense 2
- Offense 3
- Pre existing medical condition?
- Report Date
- Sex
- Specific Type of Custody/Facility
- Street Address
- Type of Custody
- Type of Offense
- Type of Offense, Other
- Version Number
- Version Type
- Were the Charges:
- Who caused the death?
- form_version
- Type of Restraint
- Under Restraint

2005 form only

- Agency County
- Custody Date NA
- Death Causer Other
- Department Type
- Entry Behavior
- Ethnicity
- Ethnicity Other
- Other Behavior
- Specify Other Behavior

2016 form only

- Exhibit any medical problems?
- Exhibit any mental health problems?
- Make suicidal statements?
- Race'''
keep_cols = []
drop_cols = set(cdr.columns)
for line in keep_text.splitlines():
    if line.startswith('- '):
        colname = line[2:]
        keep_cols.append(colname)
        drop_cols.remove(colname)

### Rename columns to be more machine friendly (lowercase, snake_case, and remove non-alphanumeric characters)

In [134]:
col_renames = {}
for c in keep_cols:
    new_name = ''.join([ch if ch.isalnum() else ' ' for ch in c.lower()])
    new_name = '_'.join(new_name.strip().split())
    col_renames[c] = new_name

In [135]:
cdr = cdr[list(col_renames.keys())]
cdr.rename(columns=col_renames, inplace=True)
cdr.head()

,age_at_time_of_death,agency_address,agency_city,agency_name,agency_zip,cdr_cdr_name,city,county,date_of_birth,date_time_of_custody_or_incident,death_date_and_time,death_location,death_location_elsewhere,entry_date_time,entry_date_time_n_a,first_name,middle_name,last_name,suffix,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,offense_1,offense_2,offense_3,pre_existing_medical_condition,report_date,sex,specific_type_of_custody_facility,street_address,type_of_custody,type_of_offense,type_of_offense_other,version_number,version_type,were_the_charges,who_caused_the_death,form_version,type_of_restraint,under_restraint,agency_county,custody_date_na,death_causer_other,department_type,entry_behavior,ethnicity,ethnicity_other,other_behavior,specify_other_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,make_suicidal_statements,race
0,40,350 W. Belknap,Fort Worth,Fort Worth Police Dept.,76102,PA05001C,Fort Worth,Tarrant,4/3/1964,1/1/05 15:04,2005-01-01 15:04:00,At the crime/arrest scene,,,1,Dino,,Gomez,,Justifiable Homicide,,Firearm,,Gunshot Wound to the Chest,"Yes, results are available",Not Applicable,Aggravated Assault,,,Not Applicable; cause of death was accidental ...,3/2/05 14:48,Male,Custody of Peace Officer during/fleeing arrest,1509 W. Hammond,Police Custody (pre-booking),,,1,ORIGINAL VERSION,Not filed at time of death,Law enforcement/correctional staff,V_2005,,No,TARRANT,0.0,,POLICE,NA,Hispanic,,0.0,,NaN,NaN,NaN,NaN
1,51,P. O. Box 1748,Austin,Travis County Sheriff's Dept.,78767,PA05002CJ,Del Valle,Travis,3/4/1953,10/20/04 20:56,2005-01-01 15:59:00,At medical facility,,10/20/04 20:56,0,Michael,Darnell,Dickson,,Natural Causes/Illness,Cardiac Arrest,Not applicable; cause of death was intoxicatio...,,Myocardial infarction,"Yes, results are available",Yes,Man/Del/Sell/Poss Controlled Substance,,,Pre-existing medical condition,3/3/05 14:07,Male,Jail - multiple occupancy cell,3614 Bill Price Road,County Jail,,,1,ORIGINAL VERSION,Filed,"Not applicable; cause of death was suicide, in...",V_2005,,No,TRAVIS,0.0,,SHERIFF,Medical,African-American,,0.0,,NaN,NaN,NaN,NaN
2,44,P.O. Box 4003,Huntsville,Texas Department Of Criminal Justice,773424003,PA05003P,Huntsville,Walker,12/3/1960,12/17/04 9:00,2005-01-02 04:35:00,At medical facility,,12/17/04 9:00,0,Darryl,Glenn,Wallace,,Natural Causes/Illness,Cardiac,Not applicable; cause of death was intoxicatio...,,Cardiac death secondary to cardiac ischemia th...,"Yes, results are available",Yes,Burglary of Habitation,,,Pre-existing medical condition,3/3/05 14:16,Male,TDCJ,21 FM 247,Penitentiary,,,1,ORIGINAL VERSION,Convicted,"Not applicable; cause of death was suicide, in...",V_2005,,No,TRAVIS,0.0,,STAGENCY,NA,African-American,,0.0,,NaN,NaN,NaN,NaN
3,21,"214 W. Nueva, Suite 331",San Antonio,San Antonio Police Dept.,78207,PA05004C,San Antonio,Bexar,11/28/1983,1/3/05 14:50,2005-01-03 15:00:00,At the crime/arrest scene,,,1,Albert,Chavez,Enriquez,,Justifiable Homicide,,Firearm,,Multiple Gunshot Wounds,"Yes, results are available",Not Applicable,Aggravated Robbery,,,Not Applicable; cause of death was accidental ...,3/3/05 14:40,Male,Custody of Peace Officer during/fleeing arrest,2102 Goliad Road,Police Custody (pre-booking),,,1,ORIGINAL VERSION,Not filed at time of death,Law enforcement/correctional staff,V_2005,,No,BEXAR,0.0,,POLICE,NA,Hispanic,,0.0,,NaN,NaN,NaN,NaN
4,26,"1200 Travis, 17th Floor",Houston,Houston Police Dept.,77002,PA05005C,Houston,Harris,12/30/1978,1/3/05 17:15,2005-01-03 17:15:00,At the crime/arrest scene,,,1,Alex,,Mendez,,Justifiable Homicide,,Firearm,,Multiple Gunshot Wounds,"Yes, results are available",Not Applicable,Traffic Violation,,,Not Applicable; cause of death was accidental ...,3/3/05 14:51,Male,Custody of Peace Officer during/fleeing arrest,3600 block of Telephone Rd,Police Custody (pre-booking),,Traffic Violation,1,ORIGINAL VERSION,Not filed at 

<div style="background-color: #008;height: 15px"></div>

## 2b. Add rows from the old master file to get BJS inpatient deaths (will dedup later)

In [136]:
# gc.drive.enable_team_drive('0ACeQWapAwOLqUk9PVA')
# titles = gc.spreadsheet_titles()
# ids = gc.spreadsheet_ids()

# cdr_id = [ids[index] for index in range(len(titles)) if titles[index] == 'reformatted_cdr_2017_master_file'][0]
# output = gc.open_by_key(cdr_id)

# wk = output[0]

# dfs = {}

# # drop empty columns or the concatenation does not work
# for sheet_name in ['Form Version 2005', 'Form Version 2016', 'Older Forms']:
#     sheet = [i for i in range(4) if output.worksheets()[i].title == sheet_name][0]
#     wk = output[sheet]
#     try:
#         dfs[sheet_name] = wk.get_as_df(include_tailing_empty = True).drop(columns = [''], axis= 1)
#     except KeyError:
#         dfs[sheet_name] = wk.get_as_df(include_tailing_empty = True)

In [137]:
datasets = dw.load_dataset(DW_PROJECT_RAW_AND_PROCESSING, force_update=True)
old_master = datasets.dataframes['reformatted_cdr_2017_master_file']
old_master['form_version'] = 'V_BJS'
print(old_master.shape)
old_master.head()

(7735, 47)


,first_name,middle_name,last_name,suffix,age_at_time_of_death,sex,date_of_birth,offense_1,offense_2,offense_3,report_date,death_date_and_time,cdr_cdr_name,agency_name,agency_address,agency_city,agency_zip,agency_county,who_caused_the_death,death_causer_other,date_time_of_custody_or_incident,custody_date_na,type_of_custody,specific_type_of_custody_facility,street_address,city,county,death_location,death_location_elsewhere,means_of_death,means_of_death_other,manner_of_death,manner_of_death_description,entry_date_time,entry_date_time_n_a,medical_cause_of_death,pre_existing_medical_condition,medical_examinor_coroner_evalution,medical_treatment,ethnicity,ethnicity_other,other_behavior,entry_behavior,were_the_charges,type_of_offense,type_of_offense_other,form_version
0,Clyde,L,Pady,NaN,80.0,Male,2024-10-01,SEXUAL ASSLT/RAPE OF CHILD,SEXUAL ASSLT/RAPE OF CHILD,SEXUAL ASSLT/RAPE OF CHILD,3/1/2005 0:00,2005-01-02,PRISON20051001,Texas Department Of Criminal Justice,NaN,NaN,NaN,NaN,NaN,NaN,5/30/1995 0:00,NaN,Penitentiary,TDCJ,NaN,Tenessee Colony,NaN,NaN,NaN,NaN,NaN,Natural Causes/Illness,Prostate/Lung Cancer,NaN,NaN,NaN,Developed condition after admission,No,Yes,Anglo or White,NaN,NaN,NaN,Convicted,child,NaN,V_BJS
1,Winston,NaN,Wilkins,NaN,61.0,Male,1943-09-27,"STOLEN VEHICLE, UNAUTHORIZED USE",0,0,3/1/2005 0:00,2005-01-02,PRISON20051002,Texas Department Of Criminal Justice,NaN,NaN,NaN,NaN,NaN,NaN,9/8/2003 0:00,NaN,Penitentiary,TDCJ,NaN,Galveston,NaN,NaN,NaN,NaN,NaN,Natural Causes/Illness,Lung Cancer,NaN,NaN,NaN,Pre-existing medical condition,No,Yes,Anglo or White,NaN,NaN,NaN,Convicted,property,NaN,V_BJS
2,Steven,NaN,Fleming,NaN,37.0,Male,1967-10-30,SEXUAL ASSLT/RAPE AGG/SEX ABUSE,0,0,3/1/2005 0:00,2005-01-04,PRISON20051004,Texas Department Of Criminal Justice,NaN,NaN,NaN,NaN,NaN,NaN,10/7/1988 0:00,NaN,Penitentiary,TDCJ,NaN,Livingston,NaN,NaN,NaN,NaN,NaN,Natural Causes/Illness,NaN,NaN,NaN,NaN,Pre-existing medical condition,No,Yes,Black or African American,NaN,NaN,NaN,Convicted,violent,NaN,V_BJS
3,James,B,Walker,NaN,61.0,Male,1943-08-27,AGG SEXUAL ASSLT,AGG KIDNAPPING,0,3/1/2005 0:00,2005-01-04,PRISON20051005,Texas Department Of Criminal Justice,NaN,NaN,NaN,NaN,NaN,NaN,4/23/2004 0:00,NaN,Penitentiary,TDCJ,NaN,Tenessee Colony,NaN,NaN,NaN,NaN,NaN,Natural Causes/Illness,Hepatocellular Cancer,NaN,NaN,NaN,Developed condition after admission,No,Yes,Black or African American,NaN,NaN,NaN,Convicted,violent,NaN,V_BJS
4,David,L,Jones,NaN,45.0,Male,1960-05-01,INDECENCY W/ A CHILD SEXUAL CONTACT,0,0,3/1/2005 0:00,2005-01-06,PRISON20051007,Texas Department Of Criminal Justice,NaN,NaN,NaN,NaN,NaN,NaN,11/8/2001 0:00,NaN,Penitentiary,TDCJ,NaN,Abilene,NaN,NaN,NaN,NaN,NaN,Natural Causes/Illness,Cardiomyopathy Heart Failure,NaN,NaN,NaN,Pre-existing medical condition,No,Yes,Anglo or White,NaN,NaN,NaN,Convicted,child,NaN,V_BJS


In [138]:
cdr.shape, old_master.shape

((9381, 57), (7735, 47))

#### There will be a few columns that the old master file doesn't have, which is fine

In [139]:
set(cdr.columns) - set(old_master.columns)

{'department_type',
 'exhibit_any_medical_problems',
 'exhibit_any_mental_health_problems',
 'make_suicidal_statements',
 'race',
 'specify_other_behavior',
 'type_of_restraint',
 'under_restraint',
 'version_number',
 'version_type'}

#### However, it shouldn't have any novel columns that the *other* data doesn't have

In [140]:
assert(len(set(old_master.columns) - set(cdr.columns)) == 0)

### Merge the "old master" file with the other CDR records

In [141]:
cdr = pd.concat([cdr, old_master])
cdr.reset_index(inplace=True, drop=True)
cdr.sort_values('form_version', inplace=True)
cdr.head()

,age_at_time_of_death,agency_address,agency_city,agency_name,agency_zip,cdr_cdr_name,city,county,date_of_birth,date_time_of_custody_or_incident,death_date_and_time,death_location,death_location_elsewhere,entry_date_time,entry_date_time_n_a,first_name,middle_name,last_name,suffix,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,offense_1,offense_2,offense_3,pre_existing_medical_condition,report_date,sex,specific_type_of_custody_facility,street_address,type_of_custody,type_of_offense,type_of_offense_other,version_number,version_type,were_the_charges,who_caused_the_death,form_version,type_of_restraint,under_restraint,agency_county,custody_date_na,death_causer_other,department_type,entry_behavior,ethnicity,ethnicity_other,other_behavior,specify_other_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,make_suicidal_statements,race
0,40.0,350 W. Belknap,Fort Worth,Fort Worth Police Dept.,76102,PA05001C,Fort Worth,Tarrant,4/3/1964,1/1/05 15:04,2005-01-01 15:04:00,At the crime/arrest scene,,,1.0,Dino,,Gomez,,Justifiable Homicide,,Firearm,,Gunshot Wound to the Chest,"Yes, results are available",Not Applicable,Aggravated Assault,,,Not Applicable; cause of death was accidental ...,3/2/05 14:48,Male,Custody of Peace Officer during/fleeing arrest,1509 W. Hammond,Police Custody (pre-booking),,,1.0,ORIGINAL VERSION,Not filed at time of death,Law enforcement/correctional staff,V_2005,,No,TARRANT,0.0,,POLICE,NA,Hispanic,,0,,NaN,NaN,NaN,NaN
3437,33.0,133 N. Riverfront Blvd.,Dallas,Dallas County Sheriff's Dept.,75207,PA14199CJ,Dallas,Dallas,3/23/1981,4/17/14 9:49,2014-04-20 08:14:00,At medical facility,,4/17/14 9:49,0.0,Alisha,,Skeats,,Natural Causes/Illness,Complications of chronic opiate use,Not applicable; cause of death was intoxicatio...,,"Complications of chronic opiate, benzodiazepin...","Yes, results are available",Yes,Theft of Service,,,Don't know,4/28/14 9:27,Female,Hospital/Infirmary,Parkland Hospital,County Jail,,property crime,1.0,ORIGINAL VERSION,Filed,"Not applicable; cause of death was suicide, in...",V_2005,,No,Dallas,0.0,,SHERIFF,NA,Anglo,,0,,NaN,NaN,NaN,NaN
3436,64.0,"2503 Lake Road, Suite 5",Huntsville,Texas Department Of Criminal Justice,77340,PA14198P,Tennessee Colony,Anderson,11/2/1949,10/6/98 0:00,2014-04-23 09:07:00,At law enforcement facility,,10/6/98 0:00,0.0,Orville,Lee,McCoy,,Natural Causes/Illness,Malignant Pleural Effusion GT Bleesing/Anemia,Not applicable; cause of death was intoxicatio...,,ESLD-Liver Disease,"No, evaluation not planned",Yes,Aggravated Sexual Assault of a Child,,,Don't know,4/25/14 15:15,Male,TDCJ,"Michael Unit, 2664 FM 2054",Penitentiary,,,1.0,ORIGINAL VERSION,Convicted,"Not applicable; cause of death was suicide, in...",V_2005,,No,Walker,0.0,,STAGENCY,NA,Anglo,,0,,NaN,NaN,NaN,NaN
3435,53.0,"2503 Lake Road, Suite 5",Huntsville,Texas Department Of Criminal Justice,77340,PA14197P,New Boston,Bowie,5/11/1960,3/19/12 0:00,2014-04-16 06:44:00,At law enforcement facility,,4/25/12 0:00,0.0,Pedro,,Juarez,Jr,Natural Causes/Illness,Cardiac arrhythmia,Not applicable; cause of death was intoxicatio...,,Sudden unexpected death in schizophrenia due t...,"Yes, results are available",Not Applicable,Aggravated Assault,,,Don't know,4/25/14 13:25,Male,TDCJ,3899 State Hwy 98,Penitentiary,,,1.0,ORIGINAL VERSION,Convicted,"Not applicable; cause of death was suicide, in...",V_2005,,No,Walker,0.0,,STAGENCY,NA,Hispanic,,0,,NaN,NaN,NaN,NaN
3434,44.0,"2503 Lake Road, Suite 5",Huntsville,Texas Department Of Criminal Justice,77340,PA14196P,Richmon,Fort Bend,10/3/1969,2/3/14 0:00,2014-04-23 13:35:00,At medical facility,,2/3/14 0:00,0.0,James,Neal,Benjamin,,Natural Causes/Illness,Thromboembolism,Not applicable; cause of death was intoxicatio...,,Acute pulmonary thromboembolism,"Yes, results are available",Not Applicable,Felony Theft ( 1yr probation) SAFPF,,,Don't know,4/25/14 11:14,Male,TDCJ,Jester 4 

<div style="background-color: #008;height: 15px"></div>

## 2c. Data is merged into one frame. Now do column type conversions

### Convert date columns

In [142]:
s1 = cdr.dtypes
convert_date_cols(cdr)
s2 = cdr.dtypes
different = s1[s1 != s2].index.tolist()
print("Changed %d cols to datetime (from some other dtype):" % len(different), different)

Converting column date_of_birth to datetime
Converting column date_time_of_custody_or_incident to datetime
Converting column death_date_and_time to datetime
Converting column entry_date_time to datetime
Replaced 1361 bad values with NaT:
Unique bad values: {'0000-00-00 00:00:00'}
Converting column report_date to datetime
Changed 4 cols to datetime (from some other dtype): ['date_of_birth', 'date_time_of_custody_or_incident', 'entry_date_time', 'report_date']


### As shown below, some float columns are in string format. We'll fix that next.

In [143]:
cdr.entry_date_time_n_a.value_counts().head()

0.0    9990
1.0    3918
Name: entry_date_time_n_a, dtype: int64

In [144]:
def float_or_nan(val):
    try:
        return float(val)
    except ValueError:
        pass
    except TypeError:
        pass
    print("- BAD VALUE (returning NaN):", val)
    return np.NAN

float_cols = [
 'age_at_time_of_death',
 'agency_zip',
 'custody_date_na',
 'entry_date_time_n_a',
 'version_number'
]

for c in float_cols:
    print("Converting", c)
    cdr[c] = cdr[c].apply(float_or_nan).astype(float)

Converting age_at_time_of_death
Converting agency_zip
- BAD VALUE (returning NaN): 78/155
Converting custody_date_na
Converting entry_date_time_n_a
Converting version_number


### Upcase string cell contents

In [145]:
upcase_strip_string_cells(cdr)

### Add a `death_date` column (without the death time)

In [146]:
cdr['death_date'] = pd.to_datetime(cdr.death_date_and_time.apply(lambda dt: datetime.date(dt.year, dt.month, dt.day)))

In [147]:
cdr_ready_to_clean = cdr.copy()

<div style="background-color: #080;height: 20px"></div>

# 3. Begin deep cleaning

In [148]:
# When we're working on modifying this file, and we need to reset
# the dataset, it's annoying to run the whole notebook, fetch from
# data.world, etc. So you can just rerun from here downward.
cdr = cdr_ready_to_clean.copy()

<div style="background-color: #008;height: 15px"></div>

## 3a.  Standardize `race` values

### First we need to merge race and ethnicity columns (the 2005 form calls it 'ethnicity', the 2016 'race')

#### Have a look at the values first

In [149]:
cdr.race.value_counts()

ANGLO OR WHITE                      1802
HISPANIC OR LATINO                  1205
BLACK OR AFRICAN AMERICAN           1180
ASIAN OR PACIFIC ISLANDER             27
OTHER                                 14
AMERICAN INDIAN OR ALASKA NATIVE       2
UNKNOWN                                1
Name: race, dtype: int64

In [150]:
cdr.ethnicity.value_counts()

ANGLO OR WHITE                      3231
BLACK OR AFRICAN AMERICAN           2275
HISPANIC OR LATINO                  2119
ANGLO                               2115
AFRICAN-AMERICAN                    1521
HISPANIC                            1433
ASIAN                                 63
OTHER/DON'T KNOW                      46
AMERICAN INDIAN/ALASKA NATIVE         26
MIDDLE EAST                           24
OTHER                                 22
NATIVE HAWAIIAN/PACIFIC ISLANDER      10
Name: ethnicity, dtype: int64

In [151]:
# Collapse the various OTHER-like values
cdr.loc[cdr.ethnicity.fillna('').str.contains('OTHER'), 'ethnicity'] = 'OTHER'

In [152]:
# When choosing the 'Other' ethnicity in the 2005 form version, there
# is a subsequent field to specify. Though clearly some of them are not
# truly 'other' ethnicities. See:
cdr.ethnicity_other.value_counts()

                       5128
0                       665
WHITE                    14
CAUCASIAN                 8
UNKNOWN                   8
WHITE NON HISPANIC        2
ANGLO & MIDDLE EAST       2
CUBAN                     2
ARABIAN                   2
SUDANESE BLACK            2
UNITED KINGDOM            1
BELIZE, BRAZILIAN         1
Name: ethnicity_other, dtype: int64

In [153]:
# Replace the nonsensical 0 with NaN
cdr.loc[cdr.ethnicity_other.astype(str) == '0', 'ethnicity_other'] = None

In [154]:
# Let's make sure nobody is filling out the "other ethnicity" column when they shouldn't...
cdr[(~(cdr.ethnicity == 'OTHER') & cdr.ethnicity_other.notnull())][['ethnicity', 'ethnicity_other']]

,ethnicity,ethnicity_other
0,HISPANIC,
3437,ANGLO,
3436,ANGLO,
3435,HISPANIC,
3434,AFRICAN-AMERICAN,
...,...,...
15203,ANGLO OR WHITE,CAUCASIAN
12333,ANGLO OR WHITE,WHITE
13088,ANGLO OR WHITE,WHITE NON HISPANIC
13021,ANGLO OR WHITE,WHITE


In [155]:
# Good. Let's transfer those specified ethnicity_other values into
# the 'ethnicity' column, so we can merge everything at once.
other_eth = (cdr.ethnicity == 'OTHER')
print('Merging %d "ethnicity_other" values into the main "ethnicity" column' % other_eth.sum())
cdr.loc[other_eth, 'ethnicity'] = cdr.ethnicity_other[other_eth]
cdr.drop('ethnicity_other', axis=1, inplace=True)

Merging 68 "ethnicity_other" values into the main "ethnicity" column


In [156]:
# Make a single 'race' column that has merged, simplified values of race or ethnicity.
race_eth_list = []
for race, eth in zip(cdr.race, cdr.ethnicity):
    # Only one of (race, eth) should be set
    assert pd.isnull(race) or pd.isnull(eth)
    if pd.isnull(race):
        if pd.isnull(eth):
            race_eth_list.append(None)
            continue
        x = eth
    else:
        x = race
    race_eth_list.append(x)

In [157]:
cdr['race'] = race_eth_list
cdr.drop('ethnicity', axis=1, inplace=True)
cdr.race.value_counts()

ANGLO OR WHITE                      5033
BLACK OR AFRICAN AMERICAN           3455
HISPANIC OR LATINO                  3324
ANGLO                               2115
AFRICAN-AMERICAN                    1521
HISPANIC                            1433
ASIAN                                 63
ASIAN OR PACIFIC ISLANDER             27
AMERICAN INDIAN/ALASKA NATIVE         26
MIDDLE EAST                           24
OTHER                                 14
NATIVE HAWAIIAN/PACIFIC ISLANDER      10
UNKNOWN                                9
WHITE                                  8
CAUCASIAN                              4
ARABIAN                                2
ANGLO & MIDDLE EAST                    2
CUBAN                                  2
AMERICAN INDIAN OR ALASKA NATIVE       2
BELIZE, BRAZILIAN                      1
WHITE NON HISPANIC                     1
UNITED KINGDOM                         1
SUDANESE BLACK                         1
Name: race, dtype: int64

### Collapse `race` values into a smaller set (white/hispanic/black/other)

In [158]:
race_before = cdr.race.copy()
race_before.name = 'race_before'

In [159]:
standardize_race_cols(cdr)
cdr.race.value_counts()

WHITE       7163
BLACK       4977
HISPANIC    4757
OTHER        181
Name: race, dtype: int64

In [160]:
cdr.groupby([race_before, 'race']).size()

race_before                       race    
AFRICAN-AMERICAN                  BLACK       1521
AMERICAN INDIAN OR ALASKA NATIVE  OTHER          2
AMERICAN INDIAN/ALASKA NATIVE     OTHER         26
ANGLO                             WHITE       2115
ANGLO & MIDDLE EAST               WHITE          2
ANGLO OR WHITE                    WHITE       5033
ARABIAN                           OTHER          2
ASIAN                             OTHER         63
ASIAN OR PACIFIC ISLANDER         OTHER         27
BELIZE, BRAZILIAN                 OTHER          1
BLACK OR AFRICAN AMERICAN         BLACK       3455
CAUCASIAN                         WHITE          4
CUBAN                             OTHER          2
HISPANIC                          HISPANIC    1433
HISPANIC OR LATINO                HISPANIC    3324
MIDDLE EAST                       OTHER         24
NATIVE HAWAIIAN/PACIFIC ISLANDER  OTHER         10
OTHER                             OTHER         14
SUDANESE BLACK                    BLACK

<div style="background-color: #008;height: 15px"></div>

## 3b. Standardize agency information

In [161]:
# Standardize agency name (so we can join/compare across datasets)
cdr['agency_name'] = cdr['agency_name'].apply(standardize_agency_name)

# Lookup county name by agency name. If this fails, fall back
# on the county specified in the form, if it exists.
cdr['agency_county'] = cdr['agency_county'].str.upper()
county_lookup = cdr['agency_name'].apply(lambda name: agency_county.get(name, np.nan))
cdr['agency_county'] = county_lookup.fillna(cdr['agency_county'])

# Manually handle one major agency
cdr.loc[cdr['agency_name'] == 'TEXAS DEPT OF CRIMINAL JUSTICE', 'agency_county'] = 'STATE'

In [162]:
# Check that we are only missing counties for a paltry few records now.
cdr[cdr['agency_county'].isnull()]['agency_name'].value_counts()

ECTOR CO SHERIFFS OFC                                       5
TEXAS JUVENILE JUSTICE DEPT OFFICE OF INSPECTOR GENERAL     3
TEXARKANA COMMUNITY COLLEGE POLICE DEPT                     2
OFFICE OF THE ATTY GENERAL                                  2
DALLASFW AIRPORT DPS                                        2
HILDAGO CO SHERIFFS OFFICE                                  2
TEST CDR AGENCY                                             2
GEO GROUP EAGLE PASS CORRECTIONAL FACILITY                  1
ELLIS CO SHERIFFS OFFICE WAYNE MCCOLLUM DETENTION CENTER    1
SUGARLAND POLICE DEPT                                       1
BEAUMONT INDEPENDENT SCHOOL DIST POLICE DEPT                1
METROPOLITAN TRANSIT AUTH OF HARRIS CO                      1
NORTHSIDE INDEPENDENT SCHOOL DIST POLICE                    1
GULF COAST VIOLENT OFFENDERS TASK FORCE                     1
RENO POLICE DEPT PARKER CO                                  1
NACONA POLICE DEPT                                          1
Name: ag

In [163]:
# Clearly 'TEST CDR AGENCY' is meant to be ignored
test_agencies = cdr['agency_name'] == 'TEST CDR AGENCY'
cdr = cdr[~test_agencies]
print("Dropping %d records from 'TEST CDR AGENCY', leaving %d records" % (test_agencies.sum(), len(cdr)))

Dropping 2 records from 'TEST CDR AGENCY', leaving 17114 records


<div style="background-color: #008;height: 15px"></div>

## 3c.  Fix death-information related columns

#### The two form versions offer slightly different options (e.g. V_2005 uses 'AT MEDICAL FACILITY' while V_2016 uses 'MEDICAL FACILITY'). Collapse values to stabilize the options across form versions, and roll up rare values into a single 'OTHER' value.

### Fix: `death_location`

In [164]:
pd.crosstab(cdr.death_location, cdr.form_version)

form_version,V_2005,V_2016,V_BJS
death_location,,,
AT LAW ENFORCEMENT FACILITY,1269,0,1134
AT MEDICAL FACILITY,3053,0,2682
AT THE CRIME/ARREST SCENE,673,0,565
DEAD ON ARRIVAL AT MEDICAL FACILITY,0,76,0
ELSEWHERE,97,0,94
"ELSEWHERE, SPECIFY",1,63,0
EN ROUTE TO BOOKING CENTER/POLICE LOCKUP,2,3,2
EN ROUTE TO MEDICAL FACILITY,54,0,50
LAW ENFORCEMENT FACILITY/BOOKING CENTER,0,695,0


In [165]:
replacements = {
    'AT MEDICAL FACILITY': 'MEDICAL FACILITY',
    'AT LAW ENFORCEMENT FACILITY': 'LAW ENFORCEMENT FACILITY',
    'AT THE CRIME/ARREST SCENE': 'CRIME/ARREST SCENE',
    'SCENE OF INCIDENT': 'CRIME/ARREST SCENE',
    'LAW ENFORCEMENT FACILITY/BOOKING CENTER': 'LAW ENFORCEMENT FACILITY',
    'DEAD ON ARRIVAL AT MEDICAL FACILITY': 'EN ROUTE TO MEDICAL FACILITY',
    'EN ROUTE TO BOOKING CENTER/POLICE LOCKUP': 'EN ROUTE TO LAW ENFORCEMENT FACILITY',
    'ELSEWHERE': OTHER_SPECIFY,
    'ELSEWHERE, SPECIFY': OTHER_SPECIFY,
}
cdr['death_location'] = cdr['death_location'].apply(lambda x: None if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr.death_location, cdr.form_version)

form_version,V_2005,V_2016,V_BJS
death_location,,,
CRIME/ARREST SCENE,674,594,565
EN ROUTE TO LAW ENFORCEMENT FACILITY,2,3,2
EN ROUTE TO MEDICAL FACILITY,54,76,50
LAW ENFORCEMENT FACILITY,1269,695,1134
MEDICAL FACILITY,3053,2798,2682
"OTHER, SPECIFY",98,63,94


### Fix: `means_of_death`

In [166]:
pd.crosstab(cdr.means_of_death, cdr.form_version)

form_version,V_2005,V_2016,V_BJS
means_of_death,,,
0,0,0,1
BATON / BLUNT INSTRUMENT,0,3,0
BLUNT INSTRUMENT,12,0,12
CONDUCTED ENERGY DEVICE (E.G. TASER),0,0,1
COULD NOT BE DETERMINED,0,0,2
DECEASED,0,0,1
DON'T KNOW,108,0,145
DON\'T KNOW,32,0,0
DRUG OVERDOSE,151,111,133


In [167]:
replacements = {
    'NOT APPLICABLE, CAUSE OF DEATH WAS ILLNESS/NATURAL CAUSE': 'NOT APPLICABLE',
    'NOT APPLICABLE; CAUSE OF DEATH WAS INTOXICATION OR ILLNESS/NATURAL CAUSES': 'NOT APPLICABLE',
    'OTHER': OTHER_SPECIFY,
    'KNIFE, CUTTING INSTRUMENT': 'KNIFE / EDGED INSTRUMENT',
    'BLUNT INSTRUMENT': 'BATON / BLUNT INSTRUMENT',
    "DON'T KNOW": 'UNKNOWN',
    "DON\\'T KNOW": 'UNKNOWN',
    'RIFLE/SHOTGUN': 'FIREARM',
}
cdr['means_of_death'] = cdr['means_of_death'].apply(lambda x:  None if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr.means_of_death, cdr.form_version)

form_version,V_2005,V_2016,V_BJS
means_of_death,,,
0,0,0,1
BATON / BLUNT INSTRUMENT,12,3,12
CONDUCTED ENERGY DEVICE (E.G. TASER),0,0,1
COULD NOT BE DETERMINED,0,0,2
DECEASED,0,0,1
DRUG OVERDOSE,151,111,133
FIREARM,892,693,774
"FIREARM, UNSPECIFIED",0,0,1
HANDGUN,0,0,100


### It seems officers sometimes misuse the 'other' option, selecting it when another category is more appropriate:

In [168]:
cdr[cdr.means_of_death == OTHER_SPECIFY]['means_of_death_other'].value_counts().head()

PENDING AUTOPSY                                                35
BLUNT FORCE TRAUMA RECEIVED DURING A MOTOR VEHICLE ACCIDENT    16
PENDING AUTOPSY RESULTS                                        15
MOTOR VEHICLE ACCIDENT                                         15
DROWNING                                                       13
Name: means_of_death_other, dtype: int64

### Let's roll all rare and 'unknown'-type values into the 'other' category. 'VEHICLE ACCIDENT' in particular is new in the 2016 form, only indicated as 'other' in older forms.

In [169]:
other_values = ['UNKNOWN', 'VEHICLE ACCIDENT', 'KNIFE / EDGED INSTRUMENT', 'BATON / BLUNT INSTRUMENT']
indices = cdr['means_of_death'].isin(other_values)
cdr.loc[indices, 'means_of_death_other'] = cdr.loc[indices, 'means_of_death']
cdr.loc[indices, 'means_of_death'] = OTHER_SPECIFY

pd.crosstab(cdr.means_of_death, cdr.form_version)

form_version,V_2005,V_2016,V_BJS
means_of_death,,,
0,0,0,1
CONDUCTED ENERGY DEVICE (E.G. TASER),0,0,1
COULD NOT BE DETERMINED,0,0,2
DECEASED,0,0,1
DRUG OVERDOSE,151,111,133
FIREARM,892,693,774
"FIREARM, UNSPECIFIED",0,0,1
HANDGUN,0,0,100
HANDGUN; RIFLE/SHOTGUN,0,0,17


### Fix: `manner_of_death` and  `manner_of_death_description`

In [170]:
pd.crosstab(cdr.form_version, cdr.manner_of_death).T

form_version,V_2005,V_2016,V_BJS
manner_of_death,,,
ACCIDENTAL,3,130,0
ACCIDENTAL INJURY CAUSED BY OTHERS,20,0,21
ACCIDENTAL INJURY TO SELF,157,0,155
ALCOHOL/DRUG INTOXICATION,291,29,284
COULD NOT BE DETERMINED,3,30,0
HOMICIDE (INCLUDES JUSTIFIABLE HOMICIDE),15,551,0
HOMICIDE BY LAW ENFORCEMENT/CORRECTIONAL STAFF,0,0,889
HOMICIDE BY OTHER,0,0,72
JUSTIFIABLE HOMICIDE,679,0,0


In [171]:
pd.crosstab(cdr.manner_of_death, cdr.manner_of_death_description.notnull())

manner_of_death_description,False,True
manner_of_death,,
ACCIDENTAL,0,133
ACCIDENTAL INJURY CAUSED BY OTHERS,20,21
ACCIDENTAL INJURY TO SELF,147,165
ALCOHOL/DRUG INTOXICATION,275,329
COULD NOT BE DETERMINED,0,33
HOMICIDE (INCLUDES JUSTIFIABLE HOMICIDE),0,566
HOMICIDE BY LAW ENFORCEMENT/CORRECTIONAL STAFF,752,137
HOMICIDE BY OTHER,61,11
JUSTIFIABLE HOMICIDE,0,679


In [172]:
replacements = {
    'NATURAL': 'NATURAL CAUSES/ILLNESS',
    'JUSTIFIABLE HOMICIDE': 'HOMICIDE',
    'HOMICIDE BY LAW ENFORCEMENT/CORRECTIONAL STAFF': 'HOMICIDE',
    'OTHER HOMICIDE': 'HOMICIDE',
    'HOMICIDE (INCLUDES JUSTIFIABLE HOMICIDE)': 'HOMICIDE',
    'ACCIDENTAL INJURY CAUSED BY OTHERS': 'ACCIDENTAL',
    'ACCIDENTAL INJURY TO SELF': 'ACCIDENTAL',
    'OTHER': OTHER_SPECIFY,
    'OTHER - SPECIFY': OTHER_SPECIFY,
}
cdr['manner_of_death'] = cdr['manner_of_death'].apply(lambda x: None if pd.isnull(x) else replacements.get(x.strip(), x))

# In past versions, "pending autopsy results" was not an option, and reports had "other"
# checked with some mention of pending autopsy in the free field. We emulate this here
# to preserve consistency across form versions.
other_values = ['PENDING AUTOPSY RESULTS', 'COULD NOT BE DETERMINED']
indices = cdr['manner_of_death'].isin(other_values)
cdr.loc[indices, 'manner_of_death_description'] = cdr.loc[indices, 'manner_of_death']
cdr.loc[indices, 'manner_of_death'] = OTHER_SPECIFY

pd.crosstab(cdr.form_version, cdr.manner_of_death).T

form_version,V_2005,V_2016,V_BJS
manner_of_death,,,
ACCIDENTAL,180,130,176
ALCOHOL/DRUG INTOXICATION,291,29,284
HOMICIDE,812,551,889
HOMICIDE BY OTHER,0,0,72
NATURAL CAUSES/ILLNESS,2885,2312,5267
"OTHER, SPECIFY",141,751,203
SUICIDE,841,456,844


### A few suicides by hanging have the wrong `means_of_death`

In [173]:
frame = cdr[(cdr.manner_of_death == 'SUICIDE') & (cdr.means_of_death != 'HANGING, STRANGULATION')]
frame = frame[(frame.medical_cause_of_death.fillna('').str.contains('HANGING')) |
              frame.manner_of_death_description.fillna('').str.contains('HANGING')]
print(len(frame))
frame[['medical_cause_of_death', 'means_of_death', 'means_of_death_other', 'manner_of_death', 'manner_of_death_description']].head()

115


,medical_cause_of_death,means_of_death,means_of_death_other,manner_of_death,manner_of_death_description
3236,HANGING,"OTHER, SPECIFY",UNKNOWN,SUICIDE,
3331,ASPHYXIABY HANGING,NOT APPLICABLE,,SUICIDE,
2745,HANGING,NOT APPLICABLE,,SUICIDE,
992,PROVISIONAL DEATH IS ASPHYXIATION BY HANGING.,NOT APPLICABLE,,SUICIDE,
1058,SUICIDE BY HANGING,NOT APPLICABLE,,SUICIDE,


In [174]:
cdr.loc[frame.index, 'means_of_death'] = 'HANGING, STRANGULATION'

# Be sure we got them all
frame = cdr[(cdr.manner_of_death == 'SUICIDE') & (cdr.means_of_death != 'HANGING, STRANGULATION')]
frame = frame[frame.medical_cause_of_death.fillna('').str.contains('HANGING')]
assert len(frame) == 0

### For a few suicides, the `manner_of_death` is 'NOT APPLICABLE', which makes no sense. We need to change these to OTHER, though (as shown below) more is going on

In [175]:
frame = cdr[(cdr.manner_of_death == 'SUICIDE') & (cdr.means_of_death == 'NOT APPLICABLE')]
print(len(frame))
frame[['means_of_death', 'means_of_death_other', 'manner_of_death', 'manner_of_death_description', 'medical_cause_of_death']].head()

47


,means_of_death,means_of_death_other,manner_of_death,manner_of_death_description,medical_cause_of_death
825,NOT APPLICABLE,,SUICIDE,,SUDDEN CARDIAC DEATH SECONDARY TO EXSANGUINATI...
1022,NOT APPLICABLE,,SUICIDE,,INCISED WOUNDS OF BOTH WRISTS AND RIGHT ARM
1094,NOT APPLICABLE,,SUICIDE,,MORPHINE AND CHLORPHENIRAMINE OVERDOSE
2239,NOT APPLICABLE,,SUICIDE,,GUNSHOT WOUNDS OF RIGHT TEMPLE
1969,NOT APPLICABLE,,SUICIDE,,INCISED WOUNDS OF LEFT ARM


In [176]:
cdr.loc[frame.index, 'means_of_death'] = OTHER_SPECIFY
assert len(cdr[(cdr.manner_of_death == 'SUICIDE') & (cdr.means_of_death == 'NOT APPLICABLE')]) == 0

### Fix: `pre_existing_medical_condition` 

In [177]:
pd.crosstab(cdr.form_version, cdr.pre_existing_medical_condition).T

form_version,V_2005,V_2016,V_BJS
pre_existing_medical_condition,,,
0,0,0,4
COULD NOT BE DETERMINED,0,2269,42
DECEASED DEVELOPED CONDITION AFTER ADMISSION,0,202,3
DEVELOPED CONDITION AFTER ADMISSION,155,0,316
DON'T KNOW,1147,0,1943
DON\'T KNOW,836,0,340
NO,0,0,3
NOT APPLICABLE,0,0,4
"NOT APPLICABLE, CAUSE OF DEATH WAS ILLNESS/NATURAL CAUSE",0,0,1


In [178]:
replacements = {
    'DECEASED DEVELOPED CONDITION AFTER ADMISSION': 'DEVELOPED CONDITION AFTER ADMISSION',
    "DON'T KNOW": 'UNKNOWN',
    "DON\\'T KNOW": 'UNKNOWN',
    'NOT APPLICABLE; CAUSE OF DEATH WAS ACCIDENTAL INJURY, INTOXICATION, SUICIDE OR HOMICIDE': 'NOT APPLICABLE',
    'COULD NOT BE DETERMINED': 'UNKNOWN',
    'PRE-EXISTING MEDICAL CONDITION': 'YES',
}
cdr['pre_existing_medical_condition'] = cdr['pre_existing_medical_condition'].apply(lambda x: None if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr.form_version, cdr.pre_existing_medical_condition).T

form_version,V_2005,V_2016,V_BJS
pre_existing_medical_condition,,,
0,0,0,4
DEVELOPED CONDITION AFTER ADMISSION,155,202,319
NO,0,0,3
NOT APPLICABLE,2072,1292,2060
"NOT APPLICABLE, CAUSE OF DEATH WAS ILLNESS/NATURAL CAUSE",0,0,1
PENDING AUTOPSY RESULTS,0,0,1
UNKNOWN,1983,2269,2327
YES,940,466,2864


### Fix: `who_caused_the_death` 

NOTE: This question is framed as follows:
* 2005 form: "If the death was an accident or homicide, who caused the death?"
* 2016 form: "If the death was an accident, homicide **or suicide**, who caused the death?" (emphasis added)

Thus, we need to:
1. Collapse near-identical values from different forms, similar to the other areas here.
1. Remove suicides from the 2016 responses, as they skew the data (see below). While we're add it, change ANY entries that are not of type homicide/suicide to have "NOT APPLICABLE" as the value.

#### Collapse near-identical values from different forms, similar to the other areas here.

In [179]:
pd.crosstab(cdr.who_caused_the_death, cdr.death_date_and_time.dt.year)

death_date_and_time,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
who_caused_the_death,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
DECEASED,22,27,28,26,24,28,10,26,38,24,25,30,0,0,0,0
DECEDENT,0,0,0,0,0,0,0,0,0,0,1,15,130,199,206,94
DON'T KNOW,6,8,12,28,46,48,41,22,16,15,33,0,0,0,0,0
DON\'T KNOW,0,0,0,0,0,0,0,0,0,2,18,27,0,0,0,0
LAW ENFORCEMENT/CORRECTIONAL PERSONNEL,0,0,0,0,0,0,0,0,2,3,4,28,86,174,191,86
LAW ENFORCEMENT/CORRECTIONAL STAFF,82,86,122,115,131,118,99,170,162,166,207,140,0,0,0,0
NOT APPLICABLE,0,0,0,0,0,0,0,0,0,0,0,123,621,845,759,566
"NOT APPLICABLE; CAUSE OF DEATH WAS SUICIDE, INTOXICATION OR ILLNESS/NATURAL CAUSES",460,520,572,505,447,425,453,476,1129,1043,1090,879,0,0,0,0


In [180]:
replacements = {
    'DECEASED': 'DECEDENT',
    "DON'T KNOW": 'UNKNOWN',
    "DON\\'T KNOW": 'UNKNOWN',
    'LAW ENFORCEMENT/CORRECTIONAL STAFF': 'LAW ENFORCEMENT/CORRECTIONAL PERSONNEL',
    'NOT APPLICABLE; CAUSE OF DEATH WAS SUICIDE, INTOXICATION OR ILLNESS/NATURAL CAUSES': 'NOT APPLICABLE',
    'OTHER DETAINEES': 'OTHER DETAINEE(S)',
    'OTHER PERSONS': 'OTHER CIVILIAN(S)',
    'ACCIDENTAL INJURY TO SELF': 'ACCIDENTAL',
    'UNKNOWN PERSON(S) CAUSED THE INJURY': 'UNKNOWN',
    'UNKNOWN WHETHER DECEDENT SUSTAINED A FATAL INJURY': 'UNKNOWN',
}
cdr['who_caused_the_death'] = cdr['who_caused_the_death'].apply(lambda x:  None if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr['who_caused_the_death'], cdr.form_version)

form_version,V_2005,V_2016,V_BJS
who_caused_the_death,,,
0,0,0,1
DECEDENT,156,638,159
LAW ENFORCEMENT/CORRECTIONAL PERSONNEL,731,555,886
NOT APPLICABLE,4015,2854,4044
OTHER CIVILIAN(S),46,19,45
OTHER DETAINEE(S),49,39,50
TAKING MEDICATION FOR HYPERTENSION-LISINOPRIL 10MG,0,0,1
UNKNOWN,153,124,170


#### Remove suicides from the 2016 responses to 'who caused the death' question, as in previous years 'who caused the death' was only asked for homicides and accidents. (see below)

In [181]:
pd.crosstab(cdr.who_caused_the_death, cdr.manner_of_death)

manner_of_death,ACCIDENTAL,ALCOHOL/DRUG INTOXICATION,HOMICIDE,HOMICIDE BY OTHER,NATURAL CAUSES/ILLNESS,"OTHER, SPECIFY",SUICIDE
who_caused_the_death,,,,,,,
0,1,0,0,0,0,0,0
DECEDENT,249,43,7,4,12,132,506
LAW ENFORCEMENT/CORRECTIONAL PERSONNEL,5,4,2071,0,0,92,0
NOT APPLICABLE,138,507,34,5,7968,644,1617
OTHER CIVILIAN(S),26,2,28,18,4,29,3
OTHER DETAINEE(S),10,0,76,45,0,7,0
TAKING MEDICATION FOR HYPERTENSION-LISINOPRIL 10MG,0,0,0,0,1,0,0
UNKNOWN,54,48,36,0,115,189,5


In [182]:
cdr.loc[~cdr.manner_of_death.isin(['HOMICIDE', 'ACCIDENTAL', OTHER_SPECIFY]), 'who_caused_the_death'] = 'NOT APPLICABLE'
pd.crosstab(cdr.who_caused_the_death, cdr.manner_of_death)

manner_of_death,ACCIDENTAL,ALCOHOL/DRUG INTOXICATION,HOMICIDE,HOMICIDE BY OTHER,NATURAL CAUSES/ILLNESS,"OTHER, SPECIFY",SUICIDE
who_caused_the_death,,,,,,,
0,1,0,0,0,0,0,0
DECEDENT,249,0,7,0,0,132,0
LAW ENFORCEMENT/CORRECTIONAL PERSONNEL,5,0,2071,0,0,92,0
NOT APPLICABLE,138,604,34,72,10464,644,2141
OTHER CIVILIAN(S),26,0,28,0,0,29,0
OTHER DETAINEE(S),10,0,76,0,0,7,0
UNKNOWN,54,0,36,0,0,189,0


### Fix `medical_examinor_coroner_evalution`

In [183]:
pd.crosstab(cdr.form_version, cdr.medical_examinor_coroner_evalution).T

form_version,V_2005,V_2016,V_BJS
medical_examinor_coroner_evalution,,,
DON'T KNOW,0,0,5
NO,0,0,1679
"NO, EVALUATION NOT PLANNED",702,893,748
"NO, EVALUATION PENDING",37,0,96
YES,0,0,695
"YES, RESULTS ARE AVAILABLE",4253,1420,4009
"YES, RESULTS PENDING",158,1916,349


In [184]:
def fix_coroner(result):
    if pd.isnull(result):
        return None
    result = result.strip()
    if result.startswith('YES'):
        return 'YES'
    elif result.startswith('NO'):
        return 'NO'
    return None

cdr['medical_examinor_coroner_evalution'] = cdr['medical_examinor_coroner_evalution'].apply(fix_coroner)
pd.crosstab(cdr.form_version, cdr.medical_examinor_coroner_evalution).T

form_version,V_2005,V_2016,V_BJS
medical_examinor_coroner_evalution,,,
NO,739,893,2523
YES,4411,3336,5053


<div style="background-color: #008;height: 15px"></div>

## 3d.  Fix other columns one-by-one

### Fix `gender` related columns

In [185]:
standardize_gender_cols(cdr)

### Fix: `were_the_charges` 

In [186]:
pd.crosstab(cdr['were_the_charges'], cdr.form_version)

form_version,V_2005,V_2016,V_BJS
were_the_charges,,,
,0,1,0
A PROBATION/PAROLE VIOLATION,0,50,1
CONVICTED,2852,2765,5278
FILED,840,506,843
NOT FILED AT TIME OF DEATH,1347,907,1351
PROBATION/PAROLE,111,0,108


In [187]:
replacements = {
    'CAPITAL MURDER': 'CONVICTED',
    'PROBATION/PAROLE': 'PROBATION/PAROLE VIOLATION',
    'A PROBATION/PAROLE VIOLATION': 'PROBATION/PAROLE VIOLATION',
}
cdr['were_the_charges'] = cdr['were_the_charges'].apply(lambda x: None if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr['were_the_charges'], cdr.form_version)

form_version,V_2005,V_2016,V_BJS
were_the_charges,,,
,0,1,0
CONVICTED,2852,2765,5278
FILED,840,506,843
NOT FILED AT TIME OF DEATH,1347,907,1351
PROBATION/PAROLE VIOLATION,111,50,109


### Fix `type_of_custody`

In [188]:
pd.crosstab(cdr['type_of_custody'], cdr.form_version)

form_version,V_2005,V_2016,V_BJS
type_of_custody,,,
COUNTY JAIL,1041,515,1058
MUNICIPAL JAIL,143,43,147
PENITENTIARY,2654,2715,5084
POLICE CUSTODY (PRE-BOOKING),1296,328,1446
PRE-CUSTODIAL USE OF FORCE,0,546,0
PRIVATE CORRECTIONAL FACILITY,16,0,0
PRIVATE FACILITY,0,82,0


In [189]:
replacements = {
    'PRE-CUSTODIAL USE OF FORCE': 'POLICE CUSTODY (PRE-BOOKING)',
    'PRIVATE CORRECTIONAL FACILITY': 'PRIVATE FACILITY',
    'COUNTY JAIL': 'JAIL - COUNTY',
    'MUNICIPAL JAIL': 'JAIL - MUNICIPAL',
    'PENITENTIARY': 'PRISON',
}
cdr['type_of_custody'] = cdr['type_of_custody'].apply(lambda x: None if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr['type_of_custody'], cdr.form_version)

form_version,V_2005,V_2016,V_BJS
type_of_custody,,,
JAIL - COUNTY,1041,515,1058
JAIL - MUNICIPAL,143,43,147
POLICE CUSTODY (PRE-BOOKING),1296,874,1446
PRISON,2654,2715,5084
PRIVATE FACILITY,16,82,0


### Fix `specific_type_of_custody_facility`

In [190]:
pd.crosstab(cdr.specific_type_of_custody_facility, cdr.form_version)

form_version,V_2005,V_2016,V_BJS
specific_type_of_custody_facility,,,
CORRECTIONAL/REHABILITATION FACILITY,33,2,32
CUSTODY OF LAW ENFORCEMENT PERSONNEL DURING/FLEEING ARREST,6,438,23
CUSTODY OF LAW ENFORCEMENT PERSONNEL SUBSEQUENT TO ARREST,2,149,5
CUSTODY OF PEACE OFFICER DURING/FLEEING ARREST,954,0,939
CUSTODY OF PEACE OFFICER SUBSEQUENT TO ARREST,246,0,241
HALFWAY HOUSE/RESTITUTION CENTER,12,0,12
HOSPITAL/INFIRMARY,307,220,308
JAIL - DAY ROOM/RECREATION AREA,25,16,26
JAIL - DETOX CELL,60,28,61


In [191]:
replacements = {
    'TDCJ, SPECIFY': 'TDCJ',
    'CUSTODY OF PEACE OFFICER DURING/FLEEING ARREST': 'CUSTODY OF LAW ENFORCEMENT PERSONNEL DURING/FLEEING ARREST',
    'CUSTODY OF PEACE OFFICER SUBSEQUENT TO ARREST': 'CUSTODY OF LAW ENFORCEMENT PERSONNEL AFTER ARREST',
    'CUSTODY OF LAW ENFORCEMENT PERSONNEL SUBSEQUENT TO ARREST': 'CUSTODY OF LAW ENFORCEMENT PERSONNEL AFTER ARREST',
    'TEXAS-JUVENILE JUSTICE DEPARTMENT - FACILITY/DETENTION CENTER, SPECIFY': 'OTHER',
    'TJPC': 'OTHER',
    'TYC': 'OTHER',
    'HALFWAY HOUSE/RESTITUTION CENTER': 'OTHER',
    'CORRECTIONAL/REHABILITATION FACILITY': 'OTHER',
    'NON-LAW ENFORCEMENT DETOX FACILITY': 'OTHER',
}
cdr['specific_type_of_custody_facility'] = cdr['specific_type_of_custody_facility'].apply(
    lambda x: x if pd.isnull(x) else replacements.get(x.strip(), x))
pd.crosstab(cdr['specific_type_of_custody_facility'], cdr.form_version)

form_version,V_2005,V_2016,V_BJS
specific_type_of_custody_facility,,,
CUSTODY OF LAW ENFORCEMENT PERSONNEL AFTER ARREST,248,149,246
CUSTODY OF LAW ENFORCEMENT PERSONNEL DURING/FLEEING ARREST,960,438,962
HOSPITAL/INFIRMARY,307,220,308
JAIL - DAY ROOM/RECREATION AREA,25,16,26
JAIL - DETOX CELL,60,28,61
JAIL - HOLDING CELL,124,47,125
JAIL - MULTIPLE OCCUPANCY CELL,383,164,385
JAIL - SINGLE CELL,360,188,359
N/A,0,250,0


### Glance at the types of custody crossed with facility

In [192]:
pd.crosstab(cdr.specific_type_of_custody_facility, cdr.type_of_custody)

type_of_custody,JAIL - COUNTY,JAIL - MUNICIPAL,POLICE CUSTODY (PRE-BOOKING),PRISON,PRIVATE FACILITY
specific_type_of_custody_facility,,,,,
CUSTODY OF LAW ENFORCEMENT PERSONNEL AFTER ARREST,16,12,612,0,3
CUSTODY OF LAW ENFORCEMENT PERSONNEL DURING/FLEEING ARREST,8,0,2352,0,0
HOSPITAL/INFIRMARY,534,9,226,6,60
JAIL - DAY ROOM/RECREATION AREA,65,0,0,0,2
JAIL - DETOX CELL,110,35,4,0,0
JAIL - HOLDING CELL,202,82,12,0,0
JAIL - MULTIPLE OCCUPANCY CELL,821,106,2,0,3
JAIL - SINGLE CELL,807,89,0,6,5
N/A,1,0,246,0,3


### `other_behavior` and `specify_other_behavior` are very annoying and misused

In [193]:
s = cdr['other_behavior'].value_counts()
print("%d records have 'other_behavior' set (%d do not)" % (s.sum(), len(cdr) - s.sum()))
print("%d unique values for 'other_behavior' - sample below:" % len(set(s)))
s.head(10)

5299 records have 'other_behavior' set (11815 do not)
5 unique values for 'other_behavior' - sample below:


0.0                                                                             4974
1.0                                                                              176
UNKNOWN                                                                            3
THREATEN HOSTAGE                                                                   2
MADE SHOOTING GESTURE WITH HAND                                                    1
PENDING TEXICOLOGY                                                                 1
UPSET OVER NOT GETTING THE METH HE THOUGHT HE PURCHASED EARLIER THAT MORNING       1
HELD HOSTAGES                                                                      1
EXIBITED ALCOHOL WITHDRAWAL                                                        1
DETOX/ALCOHOL WITHDRAWALS                                                          1
Name: other_behavior, dtype: int64

In [194]:
s = cdr['specify_other_behavior'].value_counts()
print("%d records have 'specify_other_behavior' set (%d do not)" % (s.sum(), len(cdr) - s.sum()))
print("%d unique values for 'specify_other_behavior' - sample below:" % len(set(s)))
s.head(10)

5150 records have 'specify_other_behavior' set (11964 do not)
4 unique values for 'specify_other_behavior' - sample below:


                                                                                            4974
UNKNOWN                                                                                        3
THREATEN HOSTAGE                                                                               2
FIRED SHOTS AT OFFICER                                                                         1
DETOX/ALCOHOL WITHDRAWALS                                                                      1
INJESTED LSD (4 HITS)                                                                          1
FIRED WEAPON AT OFFICERS                                                                       1
KICKED WINDOW OUT OF PATROL CAR                                                                1
THREAT TO THIRD PERSON.                                                                        1
OFFICERS REMAINED OUTSIDE DURING THE INCIDENT WITHOUT ESTABLISHING TWO-WAY COMMUNICATION       1
Name: specify_other_behavior, 

### It _appears_ that anytime other_behavior is 1.0, specify_other_behavior is set:

In [195]:
cdr[cdr['specify_other_behavior'].notnull()]['other_behavior'].fillna('(None)').value_counts()

0.0    4974
1.0     176
Name: other_behavior, dtype: int64

### Alright, we can just overwrite 'other_behavior' with 'specify_other_behavior' when the latter is given

In [196]:
behavior = []
for other, specify in zip(cdr['other_behavior'], cdr['specify_other_behavior']):
    if pd.notnull(specify):
        behavior.append(specify)
    else:
        try:
            other = float(other)
            if other == 0.0:
                behavior.append(None)
                continue
        except TypeError:
            pass
        except ValueError:
            pass
        behavior.append(other)

cdr['other_behavior'] = pd.Series(behavior, index=cdr.index)
cdr.drop('specify_other_behavior', axis=1, inplace=True)

cdr['other_behavior'].fillna('(None)').value_counts().head(10)

(None)                                                                                  11815
                                                                                         4974
UNKNOWN                                                                                     6
THREATEN HOSTAGE                                                                            4
INJESTED LSD (4 HITS)                                                                       2
ERRATIC BEHAVIOR                                                                            2
BIT MEDICAL PERSONNEL                                                                       2
STARTED KICKING PATROL VEHICLE WINDOWS ONCE THEY ARRIVED AT THE JAIL PROCESSING AREA        2
EVADING ARREST/VEHICLE                                                                      2
HELD A HOSTAGE AT GUNPOINT                                                                  2
Name: other_behavior, dtype: int64

<div style="background-color: #008;height: 15px"></div>

## 3e. Identify and drop a range of unnecessary columns

### `entry_date_time_n_a` and `custody_date_na` are useless - we can infer them from (duh) an NA value in entry_date_time or custody_date

In [197]:
cdr.groupby([cdr.entry_date_time.isnull(), cdr.entry_date_time_n_a]).size().unstack()

entry_date_time_n_a,0.0,1.0
entry_date_time,,
False,9988.0,42.0
True,NaN,3876.0


In [198]:
pd.crosstab(cdr.custody_date_na, cdr['date_time_of_custody_or_incident'].isnull())

date_time_of_custody_or_incident,False
custody_date_na,
0.0,5815
1.0,1


In [199]:
cdr.drop(['entry_date_time_n_a', 'custody_date_na'], axis=1, inplace=True)

### 'department_type' is not consistently present. Drop it.

In [200]:
pd.crosstab(cdr.department_type.notnull(), cdr.form_version)

form_version,V_2005,V_2016,V_BJS
department_type,,,
False,0,4229,7735
True,5150,0,0


In [201]:
cdr.drop('department_type', axis=1, inplace=True)

<div style="background-color: #080;height: 20px"></div>

# 4. De-duplicate (oh boy, this is a doozy)

### In theory we should able to de-dup by the record id (`cdr_cdr_name`), BUT...
1. Sometimes two records with the same record ID differ on some columns. _Usually_ the version_number can tell us which one is the latest, but not always.
1. Sometimes we'll see multiple records for the same person with different record IDs (ugh).

...so it's complicated.

### After a LOT of tinkering, I think we can catch most reasonable duplicates with the following:
  * Drop all records that are a complete, exact duplicate of another (easy step)
  * Merge all records sharing a record id, `cdr_cdr_name`
  * Merge all records with the same name and date of birth
  * Merge all records with the same name and date of death

### 4a. Get rid of totally, utterly duplicate rows

In [202]:
pure_dups = cdr.duplicated()
cdr = cdr[~pure_dups]
print("Dropping %d rows that are 100%% duplicates of another row, leaving %d rows" % (pure_dups.sum(), len(cdr)))

Dropping 141 rows that are 100% duplicates of another row, leaving 16973 rows


### 4b. Define some dedup functions to help us

In [203]:
def merge_dup_records(frame):
    '''Master merge function. Creates one record from several that are known duplicates.'''
    # Ignore BJS records (these are from and old data dump),
    # unless there is no other option.
    form_versions_seen = set(frame['form_version'])
    if 'V_BJS' in form_versions_seen and len(form_versions_seen) > 1:
        frame = frame[frame['form_version'] != 'V_BJS']
        if len(frame) == 1:
            return frame.iloc[0], 'Keeping the only non-BJS record'
    
    
    # If one record has a higher version_number than the rest, keep that one.
    # If one record has a more recent report_date than the rest, keep that one.
    max_cols = ['version_number', 'report_date']
    for c in max_cols:
        maxval = frame[c].max()  # Implicitly ignores missing values, unless only missing values exist
        if pd.notnull(maxval):
            frame = frame[frame[c] == maxval]
            if len(frame) == 1:
                return frame.iloc[0], 'Keeping the record with greatest %s' % c

    # Otherwise, there's no way to flag the "one" right record (that I know of).
    # So we gotta merge them somehow...
    merged_rec = pd.Series(index = frame.columns,
                           name = 1000000 + frame.index[0])  # Give it a new, unique index
    awk = False
    for c in frame.columns:
        notnull = frame[c][frame[c].notnull()]

        # If all records have NA for this column, leave it as NA
        if len(notnull) == 0:
            merged_rec[c] = frame[c].iloc[0]
            continue

        # Only 1 unique not-null value? Keep that one.
        if len(notnull) == 1 or len(set(notnull)) == 1:
            merged_rec[c] = notnull.iloc[0]
            continue
        
        # Are we trying to merge record IDs? That's impossible anyway,
        # let's just concatenate them.
        if c == 'cdr_cdr_name':
            merged_rec[c] = '-'.join(notnull)
            continue
        
        # Well, poop. Multiple unique values for this column.
        # Take the most popular one ¯\_(ツ)_/¯
        # (Which will just be a random one if there's a tie ¯\_(ツ)_/¯ )
        awk = True
        vc = notnull.value_counts()
        keeper = vc.index[0]
        if vc.iloc[0] > vc.iloc[1]:
            print("  > Problem with column %s, keeping the most popular value, '%s'"
                  % (c, keeper), notnull.values)
        else:
            print("  > Problem with column %s, keeping an arbitrary tied-for-most-popular value, '%s'"
                  % (c, keeper), notnull.values)
        merged_rec[c] = keeper

    merged_rec['cdr_cdr_name'] = 'MERGED-DUPLICATES-%s' % merged_rec['cdr_cdr_name']
    if awk:
        return merged_rec, 'Merged awkwardly'
    else:
        return merged_rec, 'Merged smoothly enough'

In [204]:
def dedup_cdr_by(cdr, cols):
    '''Given a cdr dataframe, and a set of columns to use to identify duplicates, dedups/merges as needed.'''
    dups = cdr[cdr.duplicated(subset=cols, keep=False)]
    if not len(dups):
        return cdr
    unmerged_frames = []
    merged_records = []
    merge_methods = []
    for _, frame in dups.groupby(cols):
        rec, meth = merge_dup_records(frame)
        unmerged_frames.append(frame)
        merged_records.append(rec)
        merge_methods.append(meth)
        if 'awkward' in meth:
            print("...awkward merge complete for records at indices", frame.index)

    return cdr.drop(dups.index).append(merged_records), unmerged_frames, merged_records, merge_methods

### 4c. Dedup!

In [205]:
dedup_rounds = [
    ['cdr_cdr_name'],
    ['first_name', 'last_name', 'date_of_birth'],
    ['first_name', 'last_name', 'death_date'],
]
all_cdrs = [cdr]
all_merged_records = []
all_unmerged_frames = []
all_merge_methods = []
for i, dr in enumerate(dedup_rounds):
    print('**** Dedup step %d: find duplicates on these columns:' % (i + 1), dr)
    vals = dedup_cdr_by(all_cdrs[-1], dr)
    new_cdr, umf, mr, mm = vals
    all_cdrs.append(new_cdr)
    all_unmerged_frames.append(umf)
    all_merged_records.append(mr)
    all_merge_methods.append(mm)
    dropping = sum(len(f) for f in umf)
    print("Dropping %d duplicates and adding %d merged records, yielding %d records" % (
        dropping, len(mr), len(new_cdr)))

**** Dedup step 1: find duplicates on these columns: ['cdr_cdr_name']


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


  > Problem with column medical_cause_of_death, keeping an arbitrary tied-for-most-popular value, 'UNKNOWN, SUBJECT WAS RELEASED TO HOSPITAL CUSTODY 7/11/2019' ['UNKNOWN, SUBJECT WAS RELEASED TO HOSPITAL CUSTODY 7/11/2019'
 'UNKNOWN, SUBJECT WAS RELEASED TO HOSPITAL CUSTODY 07/11/2019']
...awkward merge complete for records at indices Int64Index([8095, 8096], dtype='int64')
Dropping 12476 duplicates and adding 6223 merged records, yielding 10720 records
**** Dedup step 2: find duplicates on these columns: ['first_name', 'last_name', 'date_of_birth']


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launche

  > Problem with column middle_name, keeping an arbitrary tied-for-most-popular value, 'E' ['E' '']
  > Problem with column medical_cause_of_death, keeping an arbitrary tied-for-most-popular value, 'METHICILLIN RESISTANT STAPHYLOCOCCUS AUREUS BACTEREMIA, HYPOKALEMIA, AND METABOLIC TOXIC ENCEPHALOPATHY' ['METHICILLIN RESISTANT STAPHYLOCOCCUS AUREUS BACTEREMIA, HYPOKALEMIA, AND METABOLIC TOXIC ENCEPHALOPATHY'
 'RESPIRATORY FAILURE']
...awkward merge complete for records at indices Int64Index([9139, 9141], dtype='int64')
Dropping 134 duplicates and adding 66 merged records, yielding 10646 records
**** Dedup step 3: find duplicates on these columns: ['first_name', 'last_name', 'death_date']
Dropping 26 duplicates and adding 13 merged records, yielding 10633 records


In [206]:
print("Ultimately removing %d duplicate records, leaving %d" % (
    len(all_cdrs[0]) - len(all_cdrs[-1]), len(all_cdrs[-1])))
cdr = all_cdrs[-1]

Ultimately removing 6340 duplicate records, leaving 10633


### 4d. If you want to inspect the merges (what records were merged into what), use the code below

In [207]:
for rd, cols in enumerate(dedup_rounds):
    print("In round %d, there were %d record merges based on" % (rd, len(all_merged_records[rd])), cols)

In round 0, there were 6223 record merges based on ['cdr_cdr_name']
In round 1, there were 66 record merges based on ['first_name', 'last_name', 'date_of_birth']
In round 2, there were 13 record merges based on ['first_name', 'last_name', 'death_date']


In [208]:
def show_merge(merge_round, idx):
    umf = all_unmerged_frames[merge_round][idx]
    mr = all_merged_records[merge_round][idx]
    renamed = mr.copy()
    renamed.name = 'merged'
    return umf.append(renamed)

In [209]:
show_merge(2, 1)

,age_at_time_of_death,agency_address,agency_city,agency_name,agency_zip,cdr_cdr_name,city,county,date_of_birth,date_time_of_custody_or_incident,death_date_and_time,death_location,death_location_elsewhere,entry_date_time,first_name,middle_name,last_name,suffix,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,offense_1,offense_2,offense_3,pre_existing_medical_condition,report_date,sex,specific_type_of_custody_facility,street_address,type_of_custody,type_of_offense,type_of_offense_other,version_number,version_type,were_the_charges,who_caused_the_death,form_version,type_of_restraint,under_restraint,agency_county,death_causer_other,entry_behavior,other_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,make_suicidal_statements,race,death_date
10118,40.0,NaN,NaN,TEXAS DEPT OF CRIMINAL JUSTICE,NaN,PRISON20062015,BEAUMONT,NaN,1965-11-29,1996-03-04 00:00:00,2006-04-11 00:00:00,None,NaN,NaT,BENITO,NaN,CHAVEZ,NaN,NATURAL CAUSES/ILLNESS,PRELIMINARY - CONGESTIVE HEART FAILURE,None,NaN,NaN,NO,YES,MURDER INTENDS SBI CAUSING DEATH,ASSLT PUBLIC SERVANT,0,YES,2006-06-01 00:00:00,MALE,TDCJ,NaN,PRISON,VIOLENT,NaN,NaN,NaN,CONVICTED,NOT APPLICABLE,V_BJS,NaN,NaN,STATE,NaN,NaN,NaN,NaN,NaN,NaN,HISPANIC,2006-04-11
338,49.0,P.O. BOX 4003,HUNTSVILLE,TEXAS DEPT OF CRIMINAL JUSTICE,773424003.0,PA06055P,BEAUMONT,JEFFERSON,1956-11-29,1996-03-04 12:00:00,2006-04-11 17:30:00,LAW ENFORCEMENT FACILITY,,1996-03-04 12:00:00,BENITO,,CHAVEZ,,NATURAL CAUSES/ILLNESS,HEART ATTACK,NOT APPLICABLE,,CARDIAC ARREST,NO,YES,MURDER,,,YES,2006-04-14 11:02:00,MALE,TDCJ,UNKNOWN,PRISON,,,1.0,ORIGINAL VERSION,CONVICTED,NOT APPLICABLE,V_2005,,NO,STATE,,NA,,NaN,NaN,NaN,HISPANIC,2006-04-11
merged,49.0,P.O. BOX 4003,HUNTSVILLE,TEXAS DEPT OF CRIMINAL JUSTICE,773424003.0,PA06055P,BEAUMONT,JEFFERSON,1956-11-29,1996-03-04 12:00:00,2006-04-11 17:30:00,LAW ENFORCEMENT FACILITY,,1996-03-04 12:00:00,BENITO,,CHAVEZ,,NATURAL CAUSES/ILLNESS,HEART ATTACK,NOT APPLICABLE,,CARDIAC ARREST,NO,YES,MURDER,,,YES,2006-04-14 11:02:00,MALE,TDCJ,UNKNOWN,PRISON,,,1.0,ORIGINAL VERSION,CONVICTED,NOT APPLICABLE,V_2005,,NO,STATE,,NA,,NaN,NaN,NaN,HISPANIC,2006-04-11


<div style="background-color: #080;height: 20px"></div>

# 5. Add new columns, rename/reorder, otherwise finalize dataframe

<div style="background-color: #008;height: 15px"></div>

## 5a. New columns and bookkeeping columns

### Column for the time (in days) between incarceration (or incident) and death
Note: for a few of these, the death date/time is before the incarceration/incident date. If they are only one day apart, we just call it 0 and assume it was a minor error. If they are more than a day apart, clearly there was a larger error, so we use a NaN value.

In [210]:
cdr.date_time_of_custody_or_incident.dtype

dtype('<M8[ns]')

In [211]:
def get_days(dt):
    if dt.days < -1:
        return None
    elif dt.days == -1:
        return 0
    else:
        return dt.days

delta = cdr.death_date_and_time - cdr.date_time_of_custody_or_incident
print("For %d records with death date before custoday date, setting the days_from_custody_to_death to NaN" % (delta.dt.days < -1).sum())
cdr['days_from_custody_to_death'] = delta.apply(get_days)

For 24 records with death date before custoday date, setting the days_from_custody_to_death to NaN


In [216]:
cdr['suffix']

5061     
3915     
277      
7963     
7935     
       ..
1294     
762      
3972     
8904     
1596     
Name: suffix, Length: 10633, dtype: object

In [217]:
cdr['name_full'] = ''
for col in ['first_name', 'middle_name', 'last_name']:
    cdr['name_full'] = cdr['name_full'] + ' ' + cdr[col].fillna('')
cdr['name_full'] = cdr['name_full'].apply(lambda s: ' '.join(s.strip().split()))
cdr.loc[cdr['name_full'] == '', 'name_full'] = np.nan

### Tweak bookkeping columns

In [218]:
cdr['num_revisions'] = cdr['version_number'] - 1
cdr.drop(['version_type', 'version_number'], axis=1, inplace=True)

<div style="background-color: #008;height: 15px"></div>

## 5b. Rename and reorder columns sensibly

In [219]:
col_renames = {
    'first_name': 'name_first',
    'middle_name': 'name_middle',
    'last_name': 'name_last',
    'suffix': 'name_suffix',
    'cdr_cdr_name': 'record_id',
    'death_causer_other': 'who_caused_death_in_homicide_or_accident_other',
    'who_caused_the_death': 'who_caused_death_in_homicide_or_accident',
    'death_location': 'death_location_type',
    'death_location_elsewhere': 'death_location_type_other',
    'city': 'death_location_city',
    'county': 'death_location_county',
    'street_address': 'death_location_street_address',
    'entry_date_time': 'facility_entry_date_time',
    'pre_existing_medical_condition': 'death_from_pre_existing_medical_condition',
}

In [220]:
cdr.rename(columns=col_renames, inplace=True)

In [221]:
new_order = [
    # Record indexing columns
    'record_id',
    'num_revisions',
    'form_version',
    'report_date',
    'date_time_of_custody_or_incident',

    # Deceased personal information, demographics
    'name_first',
    'name_last',
    'name_middle',
    'name_suffix',
    'name_full',
    'date_of_birth',
    'age_at_time_of_death',
    'sex',
    'race',

    # Death event information
    'death_date',
    'death_date_and_time',
    'death_location_county',
    'death_location_city',
    'death_location_street_address',
    'death_location_type',
    'death_location_type_other',
    'death_from_pre_existing_medical_condition',
    'manner_of_death',
    'manner_of_death_description',
    'means_of_death',
    'means_of_death_other',
    'medical_cause_of_death',
    'medical_examinor_coroner_evalution',
    'medical_treatment',
    'days_from_custody_to_death',
    'who_caused_death_in_homicide_or_accident',
    'who_caused_death_in_homicide_or_accident_other',

    # Criminal information on deceased
    'offense_1',
    'offense_2',
    'offense_3',
    'type_of_offense',
    'type_of_offense_other',
    'were_the_charges',

    # Facility and agency information
    'facility_entry_date_time',
    'type_of_custody',
    'specific_type_of_custody_facility',
    'agency_address',
    'agency_city',
    'agency_county',
    'agency_name',
    'agency_zip',
    
    # Deceased behavior upon entry or custody
    'type_of_restraint',
    'under_restraint',
    'entry_behavior',
    'other_behavior',
    'exhibit_any_medical_problems',
    'exhibit_any_mental_health_problems',
    'make_suicidal_statements',
]

cdr = reorder_columns_and_check(cdr, new_order)

<div style="background-color: #008;height: 15px"></div>

## 5c. Final look before writing (NO CHANGES from here until writing)

In [222]:
cdr.sample(10)

,record_id,num_revisions,form_version,report_date,date_time_of_custody_or_incident,name_first,name_last,name_middle,name_suffix,name_full,date_of_birth,age_at_time_of_death,sex,race,death_date,death_date_and_time,death_location_county,death_location_city,death_location_street_address,death_location_type,death_location_type_other,death_from_pre_existing_medical_condition,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,days_from_custody_to_death,who_caused_death_in_homicide_or_accident,who_caused_death_in_homicide_or_accident_other,offense_1,offense_2,offense_3,type_of_offense,type_of_offense_other,were_the_charges,facility_entry_date_time,type_of_custody,specific_type_of_custody_facility,agency_address,agency_city,agency_county,agency_name,agency_zip,type_of_restraint,under_restraint,entry_behavior,other_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,make_suicidal_statements
11158,PRISON20074065,NaN,V_BJS,2007-12-01 00:00:00,2005-03-25 00:00:00,PEDRO,ARRIAGA,NaN,NaN,PEDRO ARRIAGA,1944-06-22,63.0,MALE,HISPANIC,2007-11-21,2007-11-21 00:00:00,NaN,BEAUMONT,NaN,None,NaN,YES,NATURAL CAUSES/ILLNESS,SEVERE ATHEROSCLEROTIC HEART DISEASE,None,NaN,NaN,YES,YES,971.0,NOT APPLICABLE,NaN,AGG SEXUAL ASSLT CHILD,0,0,CHILD,NaN,CONVICTED,NaT,PRISON,TDCJ,NaN,NaN,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13867,PRISON201221,NaN,V_BJS,2012-06-01 00:00:00,2011-04-08 00:00:00,ONEZIME,SIMON,NaN,NaN,ONEZIME SIMON,1937-09-04,75.0,MALE,WHITE,2012-04-01,2012-04-01 00:00:00,NaN,TENESSEE COLONY,NaN,None,NaN,YES,NATURAL CAUSES/ILLNESS,MYELODYSPLASTIC SYNDROME,None,NaN,NaN,NO,YES,359.0,NOT APPLICABLE,NaN,AGG SEXUAL ASSLT CHILD,0,0,CHILD,NaN,CONVICTED,NaT,PRISON,TDCJ,NaN,NaN,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1586,PA09267P,0.0,V_2005,2009-12-08 09:55:00,1994-05-04 00:00:00,BRUCE,HARRIS,,,BRUCE HARRIS,1960-10-23,49.0,MALE,BLACK,2009-11-27,2009-11-27 10:15:00,ANDERSON,TENNESSEE COLONY,2664 FM 2054,MEDICAL FACILITY,,UNKNOWN,NATURAL CAUSES/ILLNESS,BRAIN HEMORRHAGE DUE TO HIGH BLOOD PRESSURE,NOT APPLICABLE,,BRAIN HEMORRHAGE DUE TO HIGH BLOOD PRESSURE,NO,YES,5686.0,NOT APPLICABLE,,AGG ASLT,DELIVERY OF CONTROLLED SUBSTANCE-COCAINE,,,,CONVICTED,1994-05-04 00:00:00,PRISON,TDCJ,"2503 LAKE ROAD, SUITE 5",HUNTSVILLE,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,77340.0,,NO,NA,,NaN,NaN,NaN
11973,PRISON20091091,NaN,V_BJS,2009-03-01 00:00:00,2006-07-05 00:00:00,AUGUSTINE,LOPEZ,NaN,NaN,AUGUSTINE LOPEZ,1958-07-26,51.0,MALE,HISPANIC,2009-03-08,2009-03-08 00:00:00,NaN,LUBBOCK,NaN,None,NaN,YES,NATURAL CAUSES/ILLNESS,CARDIAC FAILURE,None,NaN,NaN,NO,YES,977.0,NOT APPLICABLE,NaN,DRIVING WHILE INTOXICATED 3RD OR MORE,0,0,OTHER,NaN,CONVICTED,NaT,PRISON,TDCJ,NaN,NaN,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13417,PRISON2011320,NaN,V_BJS,2011-09-01 00:00:00,2009-12-10 00:00:00,LEROY,TELLGREN,F,NaN,LEROY F TELLGREN,1934-10-10,77.0,MALE,WHITE,2011-07-17,2011-07-17 00:00:00,NaN,TENESSEE COLONY,NaN,None,NaN,YES,NATURAL CAUSES/ILLNESS,"CHRONIC OBSTRUCTIVE PULMONARY DISORDER, HEART ...",None,NaN,NaN,YES,YES,584.0,NOT APPLICABLE,NaN,MURDER,0,0,VIOLENT,NaN,CONVICTED,NaT,PRISON,TDCJ,NaN,NaN,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,PA05106CJ,0.0,V_2005,2005-06-16 11:25:00,2005-04-17 04:50:00,CECIL,JASPER,WILLIAM,,CECIL WILLIAM JASPER,1969-12-12,35.0,MALE,BLACK,2005-04-25,2005-04-25 12:25:00,HARRIS,HOUSTON,1200 BAKER STREET,MEDICAL FACILITY,,UNKNOWN,NATURAL CAUSES/ILLNESS,ASPIRATION,NOT APPLICABLE,,ASPIRATION,NO,NO,8.0,NOT APPLICABLE,,POSSESSION OF A CONTROLLED SUBSTANCE,,,,,FILED,2005-04-17 04:50:00,JAIL - COUNTY,JAIL - SINGLE CELL,1200 BAKER STREET,HOUSTON,HARRIS,HARRIS CO SHERIFFS OFFICE,77002.0,,NO,NA,,NaN,NaN,NaN
8340,19-1336-P,0.0,V_2016,2019-12-19 07:32:00,2019-11-05 00:00:00,ROBBY,WILSON,LEE,,ROBBY LEE WILSON,2065-10-09,54.0,MALE,WHITE,2019-12-10,2019

In [223]:
show_notnull_chart(cdr)

,age_at_time_of_death,agency_address,agency_city,agency_county,agency_name,agency_zip,date_of_birth,date_time_of_custody_or_incident,days_from_custody_to_death,death_date,death_date_and_time,death_from_pre_existing_medical_condition,death_location_city,death_location_county,death_location_street_address,death_location_type,death_location_type_other,entry_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,facility_entry_date_time,form_version,make_suicidal_statements,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,name_first,name_full,name_last,name_middle,name_suffix,num_revisions,offense_1,offense_2,offense_3,other_behavior,race,record_id,report_date,sex,specific_type_of_custody_facility,type_of_custody,type_of_offense,type_of_offense_other,type_of_restraint,under_restraint,were_the_charges,who_caused_death_in_homicide_or_accident,who_caused_death_in_homicide_or_accident_other
V_2005,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.996475,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.697023,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
V_2016,1.000000,1.000000,1.000000,0.994993,1.000000,0.999666,1.000000,1.000000,0.997997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.789052,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
V_BJS,0.996839,0.004741,0.004741,0.997629,1.000000,0.004741,0.939154,0.940340,0.940340,1.000000,1.000000,0.941130,0.941920,0.004741,0.004741,0.004741,0.000000,0.000000,0.000000,0.000000,0.004741,1.000000,0.000000,1.000000,0.912288,0.004741,0.000000,0.004741,0.939945,0.941920,1.000000,1.000000,1.000000,0.631766,0.001580,0.000000,0.941920,0.924141,0.918609,0.000000,0.985381,1.000000,0.941920,1.000000,0.941920,1.000000,0.939945,0.000395,0.000000,0.000000,0.941920,0.998420,0.000000


In [224]:
print(len(cdr))

10633


In [225]:
cdr.form_version.value_counts()

V_2005    5106
V_2016    2996
V_BJS     2531
Name: form_version, dtype: int64

In [226]:
pd.crosstab(cdr.death_date.apply(lambda dt: dt.year), cdr.form_version)

form_version,V_2005,V_2016,V_BJS
death_date,,,
2005,287,0,258
2006,328,0,329
2007,366,0,302
2008,338,0,344
2009,317,0,302
2010,307,0,281
2011,301,0,311
2012,347,0,343
2013,667,1,25


### One more duplication sanity check -- any CDRs with the same name since 2018?

In [227]:
f = cdr[cdr.death_date.apply(lambda dt: dt.year) >= 2018]
vc = f['name_full'].value_counts()
vc[vc > 1]

GILBERT GOMEZ      3
JOSE RIOS          2
RENE GARCIA        2
JIMMY RAY PRICE    2
JAMES GIBSON       2
Name: name_full, dtype: int64

### Looking at these cases, they seem fine. Just name coincidences.

In [228]:
f[f['name_full'] == 'RENE GARCIA']

,record_id,num_revisions,form_version,report_date,date_time_of_custody_or_incident,name_first,name_last,name_middle,name_suffix,name_full,date_of_birth,age_at_time_of_death,sex,race,death_date,death_date_and_time,death_location_county,death_location_city,death_location_street_address,death_location_type,death_location_type_other,death_from_pre_existing_medical_condition,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,days_from_custody_to_death,who_caused_death_in_homicide_or_accident,who_caused_death_in_homicide_or_accident_other,offense_1,offense_2,offense_3,type_of_offense,type_of_offense_other,were_the_charges,facility_entry_date_time,type_of_custody,specific_type_of_custody_facility,agency_address,agency_city,agency_county,agency_name,agency_zip,type_of_restraint,under_restraint,entry_behavior,other_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,make_suicidal_statements
6610,18-823-P,0.0,V_2016,2018-08-31 10:43:00,2012-09-28,RENE,GARCIA,,,RENE GARCIA,1952-04-04,66.0,MALE,HISPANIC,2018-08-27,2018-08-27 07:10:00,GALVESTON,GALVESTON,809 HARBORSIDE DRIVE,MEDICAL FACILITY,,UNKNOWN,NATURAL CAUSES/ILLNESS,,NOT APPLICABLE,,PNEUMONIA AND GASTROINTESTINAL BLEED,NO,UNKNOWN,2159.0,NOT APPLICABLE,NaN,DRIVING WHILE INTOXICATED,,,ALCOHOL / DRUG OFFENSE,,CONVICTED,2012-09-28,PRISON,TDCJ,1012 VETERANS MEMORIAL PARKWAY,HUNTSVILLE,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,77320.0,,NO,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN
6595,18-554-P,1.0,V_2016,2018-08-28 14:37:00,2012-03-07,RENE,GARCIA,,,RENE GARCIA,1970-03-19,48.0,MALE,HISPANIC,2018-06-17,2018-06-17 05:30:00,JONES,ABILENE,12017 FM 3522,MEDICAL FACILITY,,NOT APPLICABLE,SUICIDE,,"HANGING, STRANGULATION",,HANGING,YES,NO,2293.0,NOT APPLICABLE,NaN,AGGRAVATED SEXUAL ASSAULT OF A CHILD,ASSAULT PUBLIC SERVANT,,VIOLENT CRIME AGAINST PERSONS,,CONVICTED,2012-03-07,PRISON,TDCJ,1012 VETERANS MEMORIAL PARKWAY,HUNTSVILLE,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,77320.0,HANDCUFFS,"YES, MARK WHICH RESTRAINT DEVICES WERE USED",NaN,NaN,NO,NO,NO


In [229]:
f[f['name_full'] == 'JOSE RIOS']

,record_id,num_revisions,form_version,report_date,date_time_of_custody_or_incident,name_first,name_last,name_middle,name_suffix,name_full,date_of_birth,age_at_time_of_death,sex,race,death_date,death_date_and_time,death_location_county,death_location_city,death_location_street_address,death_location_type,death_location_type_other,death_from_pre_existing_medical_condition,manner_of_death,manner_of_death_description,means_of_death,means_of_death_other,medical_cause_of_death,medical_examinor_coroner_evalution,medical_treatment,days_from_custody_to_death,who_caused_death_in_homicide_or_accident,who_caused_death_in_homicide_or_accident_other,offense_1,offense_2,offense_3,type_of_offense,type_of_offense_other,were_the_charges,facility_entry_date_time,type_of_custody,specific_type_of_custody_facility,agency_address,agency_city,agency_county,agency_name,agency_zip,type_of_restraint,under_restraint,entry_behavior,other_behavior,exhibit_any_medical_problems,exhibit_any_mental_health_problems,make_suicidal_statements
6104,18-176-P,0.0,V_2016,2018-03-19 09:58:00,2017-10-19,JOSE,RIOS,,,JOSE RIOS,1964-05-02,53.0,MALE,HISPANIC,2018-02-26,2018-02-26 12:41:00,GALVESTON,GALVESTON,809 HARBORSIDE DR,MEDICAL FACILITY,,UNKNOWN,NATURAL CAUSES/ILLNESS,,NOT APPLICABLE,,CARDIAC ARREST,NO,UNKNOWN,130.0,NOT APPLICABLE,NaN,SEXUAL ASSAULT,,,VIOLENT CRIME AGAINST PERSONS,,CONVICTED,2017-10-19,PRISON,TDCJ,1012 VETERANS MEMORIAL PARKWAY,HUNTSVILLE,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,77320.0,,NO,NaN,NaN,UNKNOWN,UNKNOWN,UNKNOWN
6047,18-102-P,0.0,V_2016,2018-02-23 13:33:00,2016-05-31,JOSE,RIOS,,,JOSE RIOS,1979-02-16,38.0,MALE,HISPANIC,2018-02-12,2018-02-12 08:48:00,SWISHER,TULIA,539 S.E. 2ND STREET,MEDICAL FACILITY,,UNKNOWN,"OTHER, SPECIFY",PENDING AUTOPSY RESULTS,NOT APPLICABLE,,MASSIVE STROKE,YES,YES,622.0,NOT APPLICABLE,NaN,SEXUAL ASSAULT OF A CHILD,DWI,DWI,VIOLENT CRIME AGAINST PERSONS,,CONVICTED,2016-05-31,PRISON,TDCJ,1012 VETERANS MEMORIAL PARKWAY,HUNTSVILLE,STATE,TEXAS DEPT OF CRIMINAL JUSTICE,77320.0,,NO,NaN,NaN,NO,NO,NO


<div style="background-color: #080;height: 20px"></div>

# 6. Write

In [106]:
if os.environ.get('CLEAN_CDR_DW') != 'TRUE':
    print("Not syncing to Data.world. To do so, set CLEAN_CDR_DW to 'TRUE'")
else:
    with dw.open_remote_file(OUTPUT_DW_PROJECT, OUTPUT_FILENAME) as w:
        print("Writing to data.world:", OUTPUT_FILENAME)
        cdr.to_csv(w, index=False)
        

Not syncing to Data.world. To do so, set CLEAN_CDR_DW to 'TRUE'


In [107]:
if os.environ.get('CLEAN_CDR_S3') != 'TRUE':
    print("Not writing to s3. To do so, set CLEAN_CDR_S3 to 'TRUE'")
else:
    csv_buffer = StringIO()
    cdr.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object('tji-public-cleaned-datasets', OUTPUT_FILENAME).put(Body=csv_buffer.getvalue())


Not writing to s3. To do so, set CLEAN_CDR_S3 to 'TRUE'


<div style="background-color: #A0A;height: 20px"></div>

# 7. TODOs -- problems that still exist in the data and need work

### [TODO] `age_at_time_of_death` and `date_of_birth` can sometimes be impossible

In [108]:
cdr.age_at_time_of_death[cdr.age_at_time_of_death < 0].value_counts().sort_index()

-25.0    1
-24.0    1
-21.0    3
-19.0    2
-17.0    1
-13.0    1
-9.0     1
-1.0     5
Name: age_at_time_of_death, dtype: int64

In [109]:
print("%d records have a birth date AFTER death date" % (cdr.date_of_birth > cdr.death_date).sum())

178 records have a birth date AFTER death date


### [TODO] `type_of_offense` is a semicolon-separated list. Break this into multiple binary columns? Or reduce to only the most serious type? Unclear what to do if want to actually USE this column

In [110]:
s = cdr.type_of_offense
print("%d unique values for type_of_offense. Sample below:" % len(set(cdr.type_of_offense)))
s.value_counts().head(25)

37 unique values for type_of_offense. Sample below:


VIOLENT CRIME AGAINST PERSONS                                                       940
VIOLENT                                                                             758
PROPERTY                                                                            503
CHILD                                                                               489
DRUG                                                                                323
ALCOHOL / DRUG OFFENSE                                                              319
OTHER                                                                               306
CRIMES AGAINST CHILD(REN)                                                           253
OTHER, SPECIFY                                                                      230
CRIME AGAINST PROPERTY                                                              181
VIOLENT CRIME AGAINST PERSONS; CRIME AGAINST PROPERTY                                47
MEDICAL / MENTAL HEALTH ASSISTAN

In [111]:
s = cdr.type_of_offense_other
print("%d unique values for type_of_offense_other. Sample below:" % len(set(cdr.type_of_offense_other)))
s.value_counts().head(10)

669 unique values for type_of_offense_other. Sample below:


THEFT                  35
TRAFFIC VIOLATION      25
EVADING ARREST         14
PAROLE VIOLATION       14
FORGERY                14
CRIMINAL TRESPASS      13
ESCAPE                 13
EMERGENCY DETENTION    12
NONE                   12
MISDEMEANOR            11
Name: type_of_offense_other, dtype: int64

### [TODO] Manner of death is sometimes 'OTHER' when it should not be
Two columns detail the manner of death: `manner_of_death` and `manner_of_death_description`. A handful of rows have `manner_of_death` = `OTHER` when they should have picked `ACCIDENT` or `HOMICIDE` -- as evinced by the information in the `manner_of_death_description` column:

In [112]:
frame = cdr[(cdr.manner_of_death == OTHER_SPECIFY) & (cdr.manner_of_death_description.notnull())]
for exempt in ('PENDING', 'DETERMIN'):
    frame = frame[~frame.manner_of_death_description.str.contains(exempt)]
print(len(frame), len(cdr))
frame[['form_version', 'manner_of_death_description', 'manner_of_death', 'medical_cause_of_death']]

90 9813


,form_version,manner_of_death_description,manner_of_death,medical_cause_of_death
7141,V_2016,OFFICER INVOLVED SHOOTING,"OTHER, SPECIFY",SEE MEDICAL REPORT - SHOOTING
7259,V_2016,GUN SHOT,"OTHER, SPECIFY","MULTIPLE GUN SHOT WOUNDS OF NECK, TORSO, AND E..."
7835,V_2016,MOTOR VEHICLE CRASH / FATAL INJURIES,"OTHER, SPECIFY",UNKNOWN / FINDINGS ARE PENDING.
8047,V_2016,GUN SHOT WOUND,"OTHER, SPECIFY",GUN SHOT WOUND TO UPPER TORSO
7501,V_2016,ACUTE COCAINE AND METHAMPHETAMINE INTOXICATION...,"OTHER, SPECIFY",EXCITED DELIRIUM CAUSED BY ACUTE COCAINE AND M...
7780,V_2016,SUBJECT DIED AS A RESULT OF BEING SHOT BY MULT...,"OTHER, SPECIFY",GUNSHOT WOUNDS.
5537,V_2016,OFFICER INVOLVED SHOOTING,"OTHER, SPECIFY",PENDING AUTOPSY
5891,V_2016,OFFICER USED DEADLY FORCE. OFFICER DISCHARGED ...,"OTHER, SPECIFY",GUNSHOT WOUND TO CRANIUM.
5357,V_2016,LIGATURE AROUND NECK,"OTHER, SPECIFY",ANOXIC ENCEPHALOPATHY
6312,V_2016,ILLNESS,"OTHER, SPECIFY","UNKNOWN AT THIS TIME, AUTOPSY RESULTS NOT SUBM..."


### [TODO] (Related) Extract motor vehicle accidents into a unique type?

In [113]:
# This starter code should get it going
frame = cdr[(cdr.manner_of_death != 'ACCIDENTAL')]
frame = frame[(frame.medical_cause_of_death.fillna('').str.contains('ACCIDENT')) |
              frame.manner_of_death_description.fillna('').str.contains('ACCIDENT')]
print(len(frame))
frame[['medical_cause_of_death', 'means_of_death', 'means_of_death_other', 'manner_of_death', 'manner_of_death_description']]

44


,medical_cause_of_death,means_of_death,means_of_death_other,manner_of_death,manner_of_death_description
7586,CEREBROVASCULAR ACCIDENT (STROKE),NOT APPLICABLE,NaN,NATURAL CAUSES/ILLNESS,NaN
5600,CARDIOVASCULAR ACCIDENT,NOT APPLICABLE,NaN,NATURAL CAUSES/ILLNESS,NaN
12893,NaN,None,NaN,NATURAL CAUSES/ILLNESS,CARDIOVASCULAR ACCIDENT
12610,NaN,None,NaN,NATURAL CAUSES/ILLNESS,CARDIO VASCULAR ACCIDENT
9966,NaN,None,NaN,NATURAL CAUSES/ILLNESS,CEREBRAL VASCULAR ACCIDENT
8523,NaN,None,NaN,NATURAL CAUSES/ILLNESS,PRELIMINARY - CEREBRAL VASCULAR ACCIDENT
8443,NaN,None,NaN,NATURAL CAUSES/ILLNESS,PRELIMINARY - CEREBRAL VASCULAR ACCIDENT
6975,AUTOPSY RESULTS / HARRIS COUNTY INSTITUTE OF F...,"OTHER, SPECIFY",VEHICLE ACCIDENT,"OTHER, SPECIFY",INVOLVED IN VEHICLE CRASH (SINGLE VEHICLE VS. ...
5069,MASSIVE LEFT CEREBRAL VASCULAR ACCIDENT (STROKE),NOT APPLICABLE,NaN,NATURAL CAUSES/ILLNESS,STROKE
4265,TOXIC EFFECTS OF OXYCODONE AND DIPHENHYDRAMINE...,NOT APPLICABLE,NaN,"OTHER, SPECIFY",ACCIDENT


### [TODO] Remove `medical_examinor_coroner_evalution`? Not sure it's useful.

In [114]:
pd.crosstab(cdr.medical_examinor_coroner_evalution, cdr.form_version).sort_values(['V_2005'],ascending=False)

form_version,V_2005,V_2016,V_BJS
medical_examinor_coroner_evalution,,,
YES,4370,1644,698
NO,736,532,1681


### [TODO] Somehow blend the `entry_behavior`/`other_behavior` field from V_2005 with the `exhibit_any_medical_problems`, `exhibit_any_mental_health_problems` and `make_suicidal_statements` from V_2016

In [115]:
pd.crosstab(cdr.entry_behavior, cdr.form_version).sort_values(['V_2005'],ascending=False)

form_version,V_2005
entry_behavior,
MEDICAL,348
INTOXICATED,192
MENTAL,82
MENTAL; MEDICAL,41
INTOXICATED; MEDICAL,29
INTOXICATED; MENTAL,19
INTOXICATED; MENTAL; MEDICAL,9
MEDICAL; NA,4
INTOXICATED; NA,1


In [116]:
pd.crosstab(cdr.exhibit_any_medical_problems, cdr.form_version).sort_values(['V_2016'],ascending=False)

form_version,V_2016
exhibit_any_medical_problems,
UNKNOWN,859
NO,554
YES,502


In [117]:
pd.crosstab(cdr.exhibit_any_mental_health_problems, cdr.form_version).sort_values(['V_2016'],ascending=False)

form_version,V_2016
exhibit_any_mental_health_problems,
UNKNOWN,1072
NO,615
YES,197


In [118]:
pd.crosstab(cdr.make_suicidal_statements, cdr.form_version).sort_values(['V_2016'],ascending=False)

form_version,V_2016
make_suicidal_statements,
UNKNOWN,990
NO,746
YES,145


### [TODO] Align various binary behavioral columns across form versions so we can keep them?

In [119]:
# These columns were completely removed. Some can stay once paired off
drop_cols

{'Agency Phone',
 'Agency State',
 'Appear intoxicated (alcohol or drugs)',
 "Attempt gain possession officer's weapon",
 'Attempt to Injure Others?',
 'Barricade self or initiate standoff?',
 'Custody Type Facility',
 'Death Reason',
 'Death Weapon Other Description',
 'Decedent Display or Use Weapon Details',
 'Decedent display/use of weapons',
 'Director First Name',
 'Director Last Name',
 'Director Middle Name',
 'Director Salutation',
 'Escape or attempt to escape/flee custody',
 "Gain possession of officer's weapon",
 'Grab, hit or fight with the officer(s)',
 'Injured By',
 'Location Category',
 'Medical Treatment Description',
 'Other Agencies Respond?',
 'Other Location Category',
 'Other device, specify',
 'Other weapon, specify',
 'Physically attempt/assault officer(s)',
 'Reporter Email',
 'Reporter Name',
 'Reporter Name Original CDR',
 'Resist being handcuffed or arrested?',
 'Specify Weapon Used',
 'Status',
 'TDCJ - Specify Unit',
 'Threaten the officer(s) involved',
 